# `Nuqleon.Linq.CompilerServices`

Provides expression tree utilities such as visitors, analyzers, rewriters, evaluators, and more.

## Reference the library

### Option 1 - Use a local build

If you have built the library locally, run the following cell to load the latest build.

In [ ]:
#r "bin/Debug/net6.0/Nuqleon.Linq.CompilerServices.dll"

### Option 2 - Use NuGet packages

If you want to use the latest published package from NuGet, run the following cell.

In [ ]:
#r "nuget:Nuqleon.Linq.CompilerServices,*-*"

## (Optional) Attach a debugger

If you'd like to step through the source code of the library while running samples, run the following cell, and follow instructions to start a debugger (e.g. Visual Studio). Navigate to the source code of the library to set breakpoints.

In [ ]:
System.Diagnostics.Debugger.Launch();

## Visitors

`System.Linq.Expressions` comes with a default `ExpressionVisitor` that visits all nodes of an expression tree and invokes the `Update` method on a node if any of its children changes (as tested by an reference equality check). This library provides additional types of visitors to enable rewrites of expressions to other types, to track scope information of variables, to visit other elements of the tree (such as reflection information), etc.

There are too many visitor types to demonstrate here, so we'll start by providing a small list and zoom in to a few of them in the sections below.

* `ExpressionVisitor<TExpression>` and related types visit an `Expression` but convert it to a `TExpression`. To do so, one implements a bunch of `Make` abstract methods, e.g. `MakeBinary` given the result of recursively converting `Left`, `Right`, etc.
* `ExpressionVisitorNarrow<...>` implements a generic visitor by overriding all of the `Make` methods for statement trees (e.g. `Block`, `Loop`, etc.) as throwing `NotSupportedException`.
* `PartialExpressionVisitor<TExpression>` implements a generic visitor by overriding all of the `Make` methods as throwing `NotSupportedException`. Users can override the nodes they want to support.
* `ExpressionVisitorWithReflection` is like an `ExpressionVisitor` but it also provides virtual methods that visit the reflection objects that occur in trees, e.g. `MethodInfo` on a `Call` node.
* `CooperativeExpressionVisitor` supports dispatching to custom visitor logic when encountering a member that has a `[Visitor]` attribute applied to it.
* `ScopedExpressionVisitor<TState>` visits an expression tree while providing ways to track declaration and use sites of `ParameterExpression` nodes.

Let's zoom in to a few of these.

### `ExpressionVisitorWithReflection`

In the sample below, we harvest all of the `MethodInfo` objects that occur in an expression tree.

In [ ]:
using System.Linq.CompilerServices;
using System.Linq.Expressions;
using System.Reflection;

class HarvestMethods : ExpressionVisitorWithReflection
{
    public HashSet<MethodInfo> Methods { get; } = new();

    protected override MethodInfo VisitMethod(MethodInfo method)
    {
        Methods.Add(method);

        return method;
    }
}

Expression<Func<string, int>> f = s => s.ToLower().Substring(1, 2).Length;

var harvester = new HarvestMethods();

harvester.Visit(f);

foreach (var method in harvester.Methods)
{
    Console.WriteLine(method);
}

System.String ToLower()


System.String Substring(Int32, Int32)


### `CooperativeExpressionVisitor`

Cooperative expression visitors enable dispatching into a user-specified visitor for nodes that refer to methods, properties, fields, or constructors. The benefit of this approach is to avoid centralizing the knowledge of analysis or rewrite rules in a centralized visitor.

As an example, we'll try to optimize an expression containing calls to methods that are annotated with a cooperative visitor that knows how to perform local optimizations. 

In [ ]:
static class Sample
{
    [Visitor(typeof(AbsVisitor))]
    public static long Abs(long x) => x < 0 ? -x : x;

    private sealed class AbsVisitor : IRecursiveExpressionVisitor
    {
        public bool TryVisit(Expression expression, Func<Expression, Expression> visit, out Expression result)
        {
            var method = (MethodCallExpression)expression;

            if (method.GetArgument(0) is ConstantExpression c && c.Value is long x)
            {
                result = Expression.Constant(Sample.Abs(x));
                return true;
            }

            result = null;
            return false;
        }
    }
}

Expression<Func<long>> f = () => Sample.Abs(-42);

var visitor = new CooperativeExpressionVisitor();

Expression res = visitor.Visit(f);

Console.WriteLine(res);

() => 42


### `ScopedExpressionVisitor<TState>`

A scoped expression visitor keeps a map of `ParameterExpression` to `TState` values for each declaration site of a variable, e.g. in `LambdaExpression.Parameters`, `BlockExpression.Variables`, and `CatchBlock.Variable`. Upon encountering a use site of a variable, this state can be looked up. This utility enabled building visitors that deal with binding of variables or that analyze declared variables and their scopes.

In the code below, we'll build a scope tracking visitor to find unbound variables in an expression `(int x) => x + y`. In this example, `x` has a declaration and a use site, but `y` does only have a use site, and is thus considered unbound.

In [ ]:
class FindUnboundVariables : ScopedExpressionVisitor<ValueTuple>
{
    public HashSet<ParameterExpression> UnboundVariables { get; } = new();

    protected override ValueTuple GetState(ParameterExpression variable) => default;

    protected override Expression VisitParameter(ParameterExpression node)
    {
        if (!TryLookup(node, out var ignored))
        {
            UnboundVariables.Add(node);
        }

        return node;
    }
}

Note that we don't care about associating some value (of type `TState`) with each declared variable. Instead, we're just interested to figure out whether a variable is defined or not. As such, we use a dummy empty `ValueTuple` type. In order to figure out whether we saw a variable in a declaration site when encountering a use site, we use the `TryLookup` method in `VisitParameter`. In case we didn't find a declaration for the variable, we consider it to be unbound and add it to `UnboundVariables`.

Next, we'll craft the expression `(int x) => x + y` using the `Expression` factory methods.

In [ ]:
var x = Expression.Parameter(typeof(int), "x");
var y = Expression.Parameter(typeof(int), "y");

var expr = Expression.Lambda<Func<int, int>>(Expression.Add(x, y), x);

Console.WriteLine(expr);

x => (x + y)


Finally, let's use our utility to visit the expression and consult the `UnboundVariables` collection.

In [ ]:
var fuv = new FindUnboundVariables();

fuv.Visit(expr);

Console.WriteLine($"Unbound variables in `{expr}` = {{ {string.Join(", ", fuv.UnboundVariables)} }}");

Unbound variables in `x => (x + y)` = { y }


## `FuncletExpression`

The `FuncletExpression` is a custom `Expression` node that represents a subexpression that can be partially evaluated. Expression visitors that are aware of `FuncletExpression` nodes can retain them in an expression tree, but any other expression visitor will cause a reduction of the node to a `ConstantExpression` by triggering evaluation of the `FuncletExpression`.

As an example, consider an expression tree visitor for math operations that only retains simple unary and binary arithmetic operations, as well as constants and default values. Any other expression will be turned into a `FuncletExpression`.

First, we'll create an expression to operate on.

In [ ]:
Expression<Func<int>> f = () => Array.Empty<int>().Length * "foo".Length + 3;

var expr = f.Body;

Console.WriteLine(expr);

((ArrayLength(Empty()) * "foo".Length) + 3)


In here, `Array.Empty<int>().Length` and `"bar".Length` are expressions that cannot be evaluated by a math engine, so we'd like to reduce them to a `ConstantExpression` by causing partial evaluation at some point in time. This can be achieved by wrapping the node with a `FuncletExpression`. To do so, let's write a visitor that retains only nodes of supported types, but wraps all the other ones with a `FuncletExpression`.

In [ ]:
class ArithOnlyVisitor : ExpressionVisitor
{
    public override Expression Visit(Expression node) =>
        node switch
        {
            null => null,
            _ when node.NodeType is
                ExpressionType.Constant or
                ExpressionType.Default
                    => node,
            _ when node.NodeType is
                ExpressionType.Add or
                ExpressionType.AddChecked or
                ExpressionType.Divide or
                ExpressionType.Modulo or
                ExpressionType.Multiply or
                ExpressionType.MultiplyChecked or
                ExpressionType.Negate or
                ExpressionType.NegateChecked or
                ExpressionType.Subtract or
                ExpressionType.SubtractChecked or
                ExpressionType.UnaryPlus
                    => base.Visit(node),
            _ => FuncletExpression.Create(node)
        };
}

Now we're ready to run our expression through the visitor and observe `Array.Empty<int>().Length` and `"bar".Length` getting wrapped in a `FuncletExpression` node.

In [ ]:
var res = new ArithOnlyVisitor().Visit(expr);

Console.WriteLine(res);

((Eval(ArrayLength(Empty())) * Eval("foo".Length)) + 3)


Note that evaluation has not taken place yet. Expression visitors that are aware of `FuncletExpression` nodes can choose to defer or avoid reducing them. An example could be an arithmetic optimizer, as shown below.

In [ ]:
class ArithOptimizer : ExpressionVisitor
{
    protected override Expression VisitBinary(BinaryExpression node)
    {
        if (node.Type == typeof(int) && node.Conversion == null && node.Method == null)
        {
            if (node.NodeType is ExpressionType.Multiply or ExpressionType.MultiplyChecked)
            {
                var left = Visit(node.Left);

                if (left is ConstantExpression leftConst)
                {
                    switch ((int)leftConst.Value)
                    {
                        case 0: // 0 * right = 0
                            return leftConst;
                        case 1: // 1 * right = right
                            return Visit(node.Right);
                    }
                }

                var right = Visit(node.Right);

                if (right is ConstantExpression rightConst)
                {
                    switch ((int)rightConst.Value)
                    {
                        case 0: // left * 0 = 0
                            return rightConst;
                        case 1: // left * 1 = left
                            return left;
                    }
                }

                return node.Update(left, conversion: null, right);
            }
            else if (node.NodeType is ExpressionType.Add or ExpressionType.AddChecked)
            {
                var left = Visit(node.Left);

                if (left is ConstantExpression leftConst && (int)leftConst.Value == 0) // 0 + right = right
                {
                    return Visit(node.Right);
                }

                var right = Visit(node.Right);

                if (right is ConstantExpression rightConst && (int)rightConst.Value == 0) // left + 0 = left
                {
                    return left;
                }

                return node.Update(left, conversion: null, right);
            }

            // Omitted similar optimizations for other operations such as Divide, Modulo, Subtract, etc.
        }

        return base.VisitBinary(node);
    }
}

We've omitted many arithmetic rules here, but just enough to demonstrate the point. When the optimizer visits the following node:

```
Eval(ArrayLength(Empty())) * Eval("foo".Length)
```

it will enter the `Multiply` branch and start by visiting the `Left` node, which corresponds to:

```
Eval(ArrayLength(Empty()))
```

Visiting a `FuncletExpression` node causes partial evaluation, which in this case will result in a `ConstantExpression` with value `0`. The optimizer then detects this particular case to reduce `0 * anything` to `0`, using the rules of multiplication. As such, it avoids evaluating `"bar".Length` entirely.

**Note:** Obviously this may take away side-effects which would otherwise occur. A proper conservative optimizer for a language with side-effects would avoid making such rewrites unless it knows that `Right` has no observable side-effects. The optimizer in `Nuqleon.Linq.Expressions.Optimizers` does operate in such a manner.

In [ ]:
var opt = new ArithOptimizer().Visit(res);

Console.WriteLine(opt);

3


## Expression tree factories

The `IExpressionFactory` abstracts over the factory methods found on `Expression` and allows for custom implementations that can inject various behaviors. For example, a custom factory could return cached shared nodes for various invocations, perform additional forms of type checking, etc.

A first implementation of `IExpressionFactory` is `ExpressionFactory` which simply calls the corresponding factory methods on `Expression`.

In [ ]:
IExpressionFactory exprFactory = ExpressionFactory.Instance;

var expr = exprFactory.Add(exprFactory.Parameter(typeof(int), "x"), exprFactory.Constant(1));

Console.WriteLine(expr);

(x + 1)


Another implementation of `IExpressionFactory` is `ExpressionUnsafeFactory` which bypasses various type checks that occur in `Expression` factory methods and can be used for expression deserializers. This can provide a significant speedup. To illustrate this, let's define a small benchmark that constructs a `MethodCallExpression` which requires reflection calls to perform type checking on the object and arguments for the method being called.

To define the benchmark, we'll leverage the facilities in `Nuqleon.Time` to build stopwatches to measure time and memory allocations.

In [ ]:
using System.Time;

class MemoryClock : IClock
{
    public long Now => GC.GetAllocatedBytesForCurrentThread();
}

IStopwatch swMem = StopwatchFactory.FromClock(new MemoryClock()).Create();
IStopwatch swTime = StopwatchFactory.Diagnostics.Create();

void Benchmark(string title, Action test, int n)
{
    swMem.Restart();
    swTime.Restart();

    for (int i = 0; i < n; i++)
    {
        test();
    }

    swTime.Stop();
    swMem.Stop();

    Console.WriteLine($"{title} completed in {swTime.ElapsedMilliseconds} ms and allocated {swMem.ElapsedTicks} bytes.");
}

// The core benchmark that constructs expression trees.

var mtd = typeof(string).GetMethod(nameof(string.Substring), new[] { typeof(int), typeof(int) });

void Benchmark(string title, IExpressionFactory factory, int n)
{
    Benchmark(title, () => factory.Call(factory.Constant("bar"), mtd, factory.Constant(1), factory.Constant(2)), n);
}

Now we're ready to invoke this benchmark to compare the regular and the unsafe expression factory.

In [ ]:
IExpressionFactory defaultFactory = ExpressionFactory.Instance;
IExpressionFactory unsafeFactory = ExpressionUnsafeFactory.Instance;

Benchmark("Default", ExpressionFactory.Instance, 1_000_000);
Benchmark("Unsafe", ExpressionUnsafeFactory.Instance, 1_000_000);

Default completed in 253 ms and allocated 168000528 bytes.


Unsafe completed in 74 ms and allocated 168008560 bytes.


Note that the execution time for the unsafe factory is lower than for the regular factory which has to perform a lot of type checks. However, memory cost is identical, in part due to the `Expression` factory methods maintaining an internal cache of `ParameterInfo[]` arrays obtained from calling `GetParameters()` on the `MethodInfo` during the type checking of the `Arguments` provided to `Call`. Despite this caching, there are still many corners of the `Expression` factories where expensive operations and sometimes allocations take place.

In the context of a trusted subsystem where (a lot of) expression trees get serialized and deserialized, bypassing the type checking can be beneficial. If the original expression, prior to serialization, did type check correctly, and we can guarantee that any types and members referenced in these expressions did not change, then the expression should still type check at the point of deserialization. This is one of the optimizations that have been used in high-density Reaqtor deployments where we recover millions of expression trees containing hundreds of nodes each. The savings achieved can add up to significantly reduce recovery times, during which event processing is stalled.

Because the expression factories implement all factory methods as `virtual`, we can also derive from the factories to add additional optimizations, e.g. caching of `Constant` or `Default` nodes to reduce the number of allocations. We illustrate this below using the `Nuqleon.Memory` faciltities for function memoization.

In [ ]:
using System.Memory;

class ExpressionFactoryWithCaching : ExpressionFactory, IClearable
{
    private readonly IMemoizedDelegate<Func<object, Type, ConstantExpression>> _makeConstant;
    private readonly IMemoizedDelegate<Func<Type, DefaultExpression>> _makeDefault;

    public ExpressionFactoryWithCaching(IMemoizer memoizer)
    {
        _makeConstant = memoizer.Memoize<object, Type, ConstantExpression>((value, type) => base.Constant(value, type));
        _makeDefault = memoizer.Memoize<Type, DefaultExpression>(type => base.Default(type));
    }

    public override ConstantExpression Constant(object value) => Constant(value, value?.GetType() ?? typeof(object));

    public override ConstantExpression Constant(object value, Type type) => _makeConstant.Delegate(value, type);

    public override DefaultExpression Default(Type type) => _makeDefault.Delegate(type);

    public void Clear()
    {
        _makeConstant.Cache.Clear();
        _makeDefault.Cache.Clear();
    }
}

Now we can create an instance of `ExpressionFactoryWithCaching` and observe the effects of caching.

In [ ]:
var mem = Memoizer.Create(MemoizationCacheFactory.Unbounded);

var factory = new ExpressionFactoryWithCaching(mem);

var const1 = factory.Constant(42);
var const2 = factory.Constant(42);

Console.WriteLine(object.ReferenceEquals(const1, const2));

True


## Expression tree analysis

Various utilities are provided in this library to make the task of analyzing expression trees easier.

### Equality comparers

`ExpressionEqualityComparer` implements `IEqualityComparer<Expression>` to compare two `Expression` instances for equality, taking binding of variables and labels (used in `Goto` and `Label` expressions) into account. For example, given trees `x => x` and `y => y`, they will compare equal even though `x` and `y` are not reference equal. That is, as long as the binding of variables between use sites and definition sites is equivalent in both trees being compared, the equality requirement is met.

An example of using comparers is shown below.

In [ ]:
var eq = new ExpressionEqualityComparer();

Expression<Func<int, int, int>> e1 = (x, y) => x * y;
Expression<Func<int, int, int>> e2 = (a, b) => a * b;

Console.WriteLine(eq.Equals(e1, e1));

True


Unbound parameters do not compare equal by default.

In [ ]:
var p1 = Expression.Parameter(typeof(int), "x");
var p2 = Expression.Parameter(typeof(int), "x");

Console.WriteLine(eq.Equals(p1, p2));

False


If one wants to treat unbound parameters different, a custom `ExpressionEqualityComparator` can be built that overrides `EqualsGlobalParameter`, as shown below.

In [ ]:
class ExpressionEqualityComparatorWithGlobalVariableEqualityByName : ExpressionEqualityComparator
{
    protected override bool EqualsGlobalParameter(ParameterExpression left, ParameterExpression right) => left.Type == right.Type && left.Name == right.Name;
}

`ExpressionEqualityComparer` provides an equality compararer by wrapping a factory for `ExpressionEqualityComparator` instances. The latter are stateful because they have to keep track of environments, i.e. the variables and labels that are declarated within the tree, in order to perform binding. To use our custom `ExpressionEqualityComparatorWithGlobalVariableEqualityByName`, we can wrap it as shown below.

In [ ]:
var eq = new ExpressionEqualityComparer(() => new ExpressionEqualityComparatorWithGlobalVariableEqualityByName());

var p1 = Expression.Parameter(typeof(int), "x");
var p2 = Expression.Parameter(typeof(int), "x");

Console.WriteLine(eq.Equals(p1, p2));

True


This time around, the global parameters are considered to be equal. We use this technique in Reaqtor to compare expressions for equality prior to performing binding operations.

### Free variable scanner

We've already built a free variable scanner manually earlier in this notebook, but the library comes with such a facility built-in. As an example, consider an expression of the form `x => x + y` where `y` is unbound.

In [ ]:
var x = Expression.Parameter(typeof(int), "x");
var y = Expression.Parameter(typeof(int), "y");

var expr = Expression.Lambda<Func<int, int>>(Expression.Add(x, y), x);

Console.WriteLine(expr);

x => (x + y)


We can now use `FreeVariableScanner` to determine whether the expression contains any free variables, or to get a list of such variables. Note that the former operation is more efficient than getting all the unbound variables and performing an `Any()` or `Count() > 0` check, because we don't have to allocate a collection.

In [ ]:
Console.WriteLine($"Has free variables = {FreeVariableScanner.HasFreeVariables(expr)}");
Console.WriteLine($"Free variables = {{ {string.Join(", ", FreeVariableScanner.Scan(expr))} }}");

Has free variables = True


Free variables = { y }


### Allow list scanner

When processing expressions, it's often useful to scan them for undesirable operations, such as calls to methods that are unsafe to the hosting environment (when evaluating an expression) or constructs that cannot be translated to some target language (e.g. when writing a query provider). This library provides allow list scanners that check expressions against a given list of allowed items. Two different types are provided:

* `ExpressionTypeAllowListScanner` which checks the `Type` for every `Expression` node in a given expression tree, e.g. to make sure a tree evaluates within a certain "type domain";
* `ExpressionMemberAllowListScanner` which checks occurrences of any `MemberInfo` on any `Expression` node in a given exprssion tree, e.g. to check for methods, properties, fields, and constructors used.

Let's first have a look at these utilities.

First, we'll explore a trivial `ExpressionTypeAllowListScanner` that only allows expressions that use `int`-based operations.

In [ ]:
var typeScanner = new ExpressionTypeAllowListScanner
{
    Types = { typeof(int) }
};

var validExpr = Expression.Add(Expression.Constant(1), Expression.Constant(2));

var res = typeScanner.Visit(validExpr);

Console.WriteLine(res);

(1 + 2)


In case the tree contains operations involving other types, it gets rejected.

In [ ]:
var invalidExpr = Expression.Add(Expression.Constant(1), Expression.Property(Expression.Constant("foo"), nameof(string.Length)));

var res = typeScanner.Visit(invalidExpr);

System.NotSupportedException: Expression '"foo"' uses 'string' which is not allowed.
   at System.Linq.CompilerServices.ExpressionTypeAllowListScannerBase.ResolveExpression[T](T expression, Type type, Func`2 visit) in D:\Projects\Reaqtive\reaqtor\Nuqleon\Core\LINQ\Nuqleon.Linq.CompilerServices\Expressions\Analysis\ExpressionTypeAllowListScannerBase.cs:line 59
   at System.Linq.CompilerServices.ExpressionTypeAllowListScannerBase.Visit(Expression node) in D:\Projects\Reaqtive\reaqtor\Nuqleon\Core\LINQ\Nuqleon.Linq.CompilerServices\Expressions\Analysis\ExpressionTypeAllowListScannerBase.cs:line 33
   at System.Linq.Expressions.ExpressionVisitor.VisitMember(MemberExpression node)
   at System.Linq.Expressions.MemberExpression.Accept(ExpressionVisitor visitor)
   at System.Linq.Expressions.ExpressionVisitor.Visit(Expression node)
   at System.Linq.CompilerServices.ExpressionTypeAllowListScannerBase.Visit(Expression node) in D:\Projects\Reaqtive\reaqtor\Nuqleon\Core\LINQ\Nuqleon.Linq.Compile

In case it's desirable to handle expressions that don't pass the allowlist rules, one can override `ResolveExpression` as shown below.

In [ ]:
class MyExpressionTypeAllowListScanner : ExpressionTypeAllowListScanner
{
    protected override Expression ResolveExpression<T>(T expression, Type type, Func<T, Expression> visit) => FuncletExpression.Create(expression);
}

var myTypeScanner = new MyExpressionTypeAllowListScanner
{
    Types = { typeof(int) }
};

var res = myTypeScanner.Visit(invalidExpr);

Console.WriteLine(res);

(1 + Eval("foo").Length)


In this example we use the sledgehammer of wrapping the rejected expression in a funclet, which will cause partial evaluation down the line. For example, prior to submitting an expression to a remote service, partial evaluation of any unsupported constructs can take place locally. A better example may be using a member-based allow list scanner, as shown below.

In [ ]:
var memberScanner = new ExpressionMemberAllowListScanner
{
    Members =
    {
        typeof(Math).GetMethod(nameof(Math.Abs), new[] { typeof(int) })
    }
};

Expression<Func<int, int>> f = x => Math.Abs(x + 1);

var res = memberScanner.Visit(f);

Console.WriteLine(res);

x => Abs((x + 1))


Here, we allow uses of `Math.Abs(int)`, but nothing else. Alternatively, we could have added `typeof(Abs)` to the `DeclaringTypes` collection on the scanner, to allow all members on `Math`.

Now, let's consider a more complex expression that refers to a variable in the outer scope. This causes the creation of a closure, which will manifest itself in the expression tree as a field lookup using a `MemberExpression`.

In [ ]:
int y = 1;
Expression<Func<int, int>> fWithClosure = x => Math.Abs(x + y);

Console.WriteLine(fWithClosure);

x => Abs((x + value(Submission#30).y))


This field lookup on the compiler-generated closure object is not present in the allow list, so the scanner will reject it.

In [ ]:
var res = memberScanner.Visit(fWithClosure);

System.NotSupportedException: Expression '__c0.y' uses 'int Submission#30.y' which is not allowed.
   at System.Linq.CompilerServices.ExpressionMemberAllowListScannerBase.ResolveExpression[T](T expression, MemberInfo member, Func`2 visit) in D:\Projects\Reaqtive\reaqtor\Nuqleon\Core\LINQ\Nuqleon.Linq.CompilerServices\Expressions\Analysis\ExpressionMemberAllowListScannerBase.cs:line 156
   at System.Linq.CompilerServices.ExpressionMemberAllowListScannerBase.CheckExpression[T](T expression, MemberInfo member, Func`2 visit) in D:\Projects\Reaqtive\reaqtor\Nuqleon\Core\LINQ\Nuqleon.Linq.CompilerServices\Expressions\Analysis\ExpressionMemberAllowListScannerBase.cs:line 96
   at System.Linq.CompilerServices.ExpressionMemberAllowListScannerBase.VisitMember(MemberExpression node) in D:\Projects\Reaqtive\reaqtor\Nuqleon\Core\LINQ\Nuqleon.Linq.CompilerServices\Expressions\Analysis\ExpressionMemberAllowListScannerBase.cs:line 58
   at System.Linq.Expressions.MemberExpression.Accept(ExpressionVisi

We can now build a custom scanner type that uses a funclet to trigger partial evaluation of unsupported nodes. In this case, it will cause reduction of the closure access to a `ConstantExpression`.

In [ ]:
class MyExpressionMemberAllowListScanner : ExpressionMemberAllowListScanner
{
    protected override Expression ResolveExpression<T>(T expression, MemberInfo member, Func<T, Expression> visit) => FuncletExpression.Create(expression);
}

var myMemberScanner = new MyExpressionMemberAllowListScanner
{
    Members =
    {
        typeof(Math).GetMethod(nameof(Math.Abs), new[] { typeof(int) })
    }
};

var res = myMemberScanner.Visit(fWithClosure);

Console.WriteLine(res);

x => Abs((x + Eval(value(Submission#30).y)))


To illustrate the partial evaluation carried out by the `FuncletExpression`, we can simply run an `ExpressionVisitor` over the expression, which will trigger calls to `Reduce`.

In [ ]:
class ReduceExpressionVisitor : ExpressionVisitor
{
}

var reduced = new ReduceExpressionVisitor().Visit(res);

Console.WriteLine(reduced);

x => Abs((x + 1))


More advanced uses of type-based and member-based allow list scanners can be constructed by deriving from the `ExpressionTypeAllowListScannerBase` and `ExpressionTypeAllowMemberScannerBase` classes instead. These provide `Check` methods that are used to check whether a type or member is allowed to be used. These predicates can be arbitrary rather than type-based. For example, one could allow any type or member in a particular assembly.

## Expression tree rewriters

This library also provides a whole plethora of expression tree rewriting utilities.

### Alpha renaming

Lambda calculus has the notion of alpha renaming whereby the names of variables get changed in a way that doesn't inadvertently changes the bindings of variable. For example, consider an expression `x => y => x + y`. In here, we can change the name of `x` to `a` but not to `y` because it'd cause the binding to change to the `y` declared on the inner lambda expression.

This library provides an implementation of alpha renaming using the `AlphaRenamer.EliminateNameConflicts` static method, as shown below.

In [ ]:
var x0 = Expression.Parameter(typeof(int), "x");
var x1 = Expression.Parameter(typeof(int), "x");

var ambiguousNames1 = Expression.Lambda(Expression.Lambda(x0, x0), x0);
var ambiguousNames2 = Expression.Lambda(Expression.Lambda(x0, x1), x1);

Console.WriteLine(ambiguousNames1);
Console.WriteLine(ambiguousNames2);

x => x => x


x => x => x


Both functions have different meanings. The first one is equivalent to `_ => x => x`, because the use site of `x` in the inner lambda binds to the declaration site of `x` as a parameter on the inner lambda. The second one is equivalent to `x => _ => x`.

Alpha renaming gets rid of the syntactic ambiguity by creating new `ParameterExpression` nodes with unique names.

In [ ]:
var uniqueNames1 = AlphaRenamer.EliminateNameConflicts(ambiguousNames1);
var uniqueNames2 = AlphaRenamer.EliminateNameConflicts(ambiguousNames2);

Console.WriteLine(uniqueNames1);
Console.WriteLine(uniqueNames2);

x => x0 => x0


x => x0 => x


### Beta reduction

Computation in lambda calculus is driven by beta reduction. Given an invocation of a lambda expression, a reduction can be made by substituting the arguments used in the invocation for the parameters of the lambda expression. For example:

```csharp
(x => x + 1)(2)
```

can be reduced to `2 + 1` by substituting `2` for `x` in the body of the lambda expression. That is, arguments get inlined in the lambda body.

Let's first translate this example to expression trees, as follows:

In [ ]:
Expression<Func<int, int>> f = x => x + 1;

var expr = Expression.Invoke(f, Expression.Constant(2));

Console.WriteLine(expr);

Invoke(x => (x + 1), 2)


We can now use the `BetaReducer` in this library to reduce the expression to `2 + 1`, as shown below.

In [ ]:
var reduced = BetaReducer.Reduce(expr);

Console.WriteLine(reduced);

(2 + 1)


The call to `Reduce` does find any `Invoke(Lambda(...), ...)` structure in the given expression tree and attempts to reduce it. Given that beta reduction is a very essential technique employed by Reaqtor to bind reactive query expressions, we'll build a slightly bigger sample below that illustrates binding and reduction.

First, let's build a little catalog of functions for a simple math engine. We'll start by defining some operations such as `Abs` and `Pow` and put them in a dictionary.

In [ ]:
var functions = new Dictionary<string, Expression>
{
    { "abs", (Expression<Func<double, double>>)(x => Math.Abs(x)) },
    { "pow", (Expression<Func<double, double, double>>)((x, y) => Math.Pow(x, y)) },
};

The goal is for users to be able to submit expressions that refer to the functions by name using unbound `ParameterExpression` nodes. For example:

In [ ]:
var expr = Expression.Invoke(Expression.Parameter(typeof(Func<double, double>), "abs"), Expression.Constant(-2.0));

Console.WriteLine(expr);

Invoke(abs, -2)


Reaqtor uses this technique to normalize expressions submitted by client libraries. Rather than doing so for math operations, it does this for reactive query operators such as `Where`, `Select`, `Window`, etc. The normalization step takes a friendly user expression such as `xs.Where(x => x > 0)` to an expression that is no longer bound to a concrete client-side `MethodInfo` for `Where`. In our simplified math example shown here, one can image a client library that allows the user to write `calculator.Abs(-2.0)`, which gets normalized to `Invoke(Abs, -2.0)` where `Abs` is an unbound parameter expression. The calculator engine then binds the `Abs` function to a concrete implementation, e.g. using `Math.Abs`.

To build a binder, we'll leverage the `FreeVariableScanner` to find all unbound parameters in the expression submitted by the user, and then build a binding expression around it. Let's do these things one step at a time and start by scanning the free variables.

In [ ]:
var freeVars = FreeVariableScanner.Scan(expr);

Console.WriteLine(string.Join(", ", freeVars));

abs


Next, we can perform a technique called *lambda lifting* to wrap the user expression in a lambda that has all of the unbound variables as parameters. This is shown below.

In [ ]:
var lambda = Expression.Lambda(expr, freeVars);

Console.WriteLine(lambda);

abs => Invoke(abs, -2)


To perform binding, we can now look up all of the unbound parameters in our `functions` registry and construct an `InvocationExpression` around the lambda that was built above.

In [ ]:
var bound = Expression.Invoke(lambda, freeVars.Select(freeVar => functions[freeVar.Name]));

Console.WriteLine(bound);

Invoke(abs => Invoke(abs, -2), x => Abs(x))


Note that expression factories perform rigorous type checking, so if the user's expression is trying to invoke `abs` using `int` operands, the binding step above would fail. Similarly, binding would fail if we don't find a particular variable in the `function` registry.

At this point, we could already evaluate the user's expression. For this, we'll make use of the `Evaluate` extension method provided by this library (discussed further on in this notebook).

In [ ]:
Console.WriteLine(bound.Evaluate<double>());

2


However, this can be rather inefficient because the compiled expression tree can be simplified by applying beta reduction steps first. This can also help with debugging, size of expressions stored, etc. So, let's give beta reduction a try.

In [ ]:
var reduced = BetaReducer.Reduce(bound);

Console.WriteLine(reduced);

Invoke(abs => Invoke(abs, -2), x => Abs(x))


This didn't work because the `BetaReducer` is very conservative when it comes to inlining non-trivial expressions which may cause reordering of side-effects. To override this behavior, we can use an overload of `Reduce`, as shown below.

In [ ]:
var reduced = BetaReducer.Reduce(bound, BetaReductionNodeTypes.Unrestricted, BetaReductionRestrictions.None);

Console.WriteLine(reduced);

Invoke(x => Abs(x), -2)


A detailed description of these parameters is outside the scope of this notebook, but a little brainteaser is shown below to point out the potential evils of performing beta reduction.

```csharp
(s => Console.ReadLine() + s + s)(Console.ReadLine())
```

When evaluating this code, the user will get two prompts to input a string. Say the user enters `bar` and `foo`, then the result will be `foobarbar`. If "naive" unrestricted beta reduction is carried out, we can end up with the following:

```csharp
Console.ReadLine() + Console.ReadLine() + Console.ReadLine()
```

which prompts the user for three inputs. If the user enters `bar`, `foo`, and `qux`, the result will be `barfooqux`.

Continuing with our running example, note that the resutling expression is not fully reduced yet. In particular, we still have an `InvocationExpression` node left in the tree. To cause further reduction, we can use the `ReduceEager` method which carries out a *fixed point* loop around the core beta reduction logic. That is, it keeps applying beta reduction until the tree no longer changes. An example is shown below.

In [ ]:
var reduced = BetaReducer.ReduceEager(bound, BetaReductionNodeTypes.Unrestricted, BetaReductionRestrictions.None, throwOnCycle: true);

Console.WriteLine(reduced);

Abs(-2)


In many cases, binding and beta reduction needs to be carried out in a *fixed loop* algorithm, because definitions may refer to other definitions. As an example, consider adding a `GetRadius` function that's built on top of `Pow` and `Sqrt`. Let's add a few definitions to our `functions` registry.

In [ ]:
functions.Add("sqrt", (Expression<Func<double, double>>)(x => Math.Sqrt(x)));

var x = Expression.Parameter(typeof(double), "x");
var y = Expression.Parameter(typeof(double), "y");

var sqrt = Expression.Parameter(typeof(Func<double, double>), "sqrt");
var pow = Expression.Parameter(typeof(Func<double, double, double>), "pow");
var two = Expression.Constant(2.0);

var getRadius = Expression.Lambda(Expression.Invoke(sqrt, Expression.Add(Expression.Invoke(pow, x, two), Expression.Invoke(pow, y, two))), x, y);

functions.Add("getRadius", getRadius);

Let's now create a reusable `Bind` function, encapsulating the logic we used before, but adding a `bool` return type to indicate whether any binding took place.

In [ ]:
bool Bind(Expression expr, out Expression bound)
{
    var freeVars = FreeVariableScanner.Scan(expr);

    if (freeVars.Count() == 0)
    {
        bound = expr;
        return false;
    }

    var lambda = Expression.Lambda(expr, freeVars);
    
    bound = Expression.Invoke(lambda, freeVars.Select(freeVar => functions[freeVar.Name]));
    return true;
}

Let's also encapsulate our beta reduction logic in a `Reduce` method.

In [ ]:
static Expression Reduce(Expression expr)
{
    var reduced = BetaReducer.ReduceEager(expr, BetaReductionNodeTypes.Unrestricted, BetaReductionRestrictions.None, throwOnCycle: true);
    return reduced;
}

To show the effects of a single pass of binding and reduction on a user expression that refers to `GetRadius`, we'll go ahead and build a such an expression.

In [ ]:
var expr = Expression.Invoke(Expression.Parameter(typeof(Func<double, double, double>), "getRadius"), Expression.Constant(3.0), Expression.Constant(4.0));

A single turn of the binding and reduction crank yields the following result.

In [ ]:
if (Bind(expr, out var bound))
{
    var res = Reduce(bound);
    Console.WriteLine(res);
}

Invoke(sqrt, (Invoke(pow, 3, 2) + Invoke(pow, 4, 2)))


This expression cannot be evaluated yet because it now refers to `Sqrt` and `Pow`. To get to the point of evaluating the expression, we need to continue binding until no more binding steps are necessary. This can be achieved using a simple loop.

In [ ]:
Expression BindFully(Expression expr)
{
    while (Bind(expr, out expr))
    {
        expr = Reduce(expr);
    }

    return expr;
}

var res = BindFully(expr);

Console.WriteLine(res);

Sqrt((Pow(3, 2) + Pow(4, 2)))


Now the expression is fully bound to `Math.*` methods, and we can go ahead an evaluate it.

In [ ]:
Console.WriteLine(res.Evaluate<double>());

5


### Eta converter

A final lambda calculus construct is eta conversion, which simplifies an expression of the form `(x => f(x))` to `f`, by taking away the redundant lambda "abstraction" whose body contains an invocation "application". The result is a simpler, more compact, expression. This library supports this form of simplifying expressions through `EtaConverter.Convert`.

In [ ]:
var x = Expression.Parameter(typeof(int), "x");
var f = Expression.Parameter(typeof(Func<int, int>), "f");

var expr = Expression.Lambda(Expression.Invoke(f, x), x);

Console.WriteLine(expr);

var simpler = EtaConverter.Convert(expr);

Console.WriteLine(simpler);

x => Invoke(f, x)


f


### Compiler-generated name elimination

The C# compiler emits compiler-generated names for certain lambda expression parameters, e.g. in the context of `let` clauses that introduce so-called "transparent identifiers". An example is shown below using `IQueryable<T>`.

In [ ]:
var res = from x in new[] { 1, 2, 3 }.AsQueryable()
          let y = x + 1
          let z = y * 2
          select x + y - z;

Console.WriteLine(res.Expression);

System.Int32[].Select(x => new <>f__AnonymousType0#51`2(x = x, y = (x + 1))).Select(<>h__TransparentIdentifier0 => new <>f__AnonymousType1#51`2(<>h__TransparentIdentifier0 = <>h__TransparentIdentifier0, z = (<>h__TransparentIdentifier0.y * 2))).Select(<>h__TransparentIdentifier1 => ((<>h__TransparentIdentifier1.<>h__TransparentIdentifier0.x + <>h__TransparentIdentifier1.<>h__TransparentIdentifier0.y) - <>h__TransparentIdentifier1.z))


The resulting expression tree shows quite verbose compiler-generated identifiers which can hamper debuggability. Using the `CompilerGeneratedNameEliminator.Prettify` utility we can rewrite the expression using simpler identifiers.

In [ ]:
var pretty = CompilerGeneratedNameEliminator.Prettify(res.Expression);

Console.WriteLine(pretty);

System.Int32[].Select(x => new <>f__AnonymousType0#51`2(x = x, y = (x + 1))).Select(t => new <>f__AnonymousType1#51`2(<>h__TransparentIdentifier0 = t, z = (t.y * 2))).Select(t => ((t.<>h__TransparentIdentifier0.x + t.<>h__TransparentIdentifier0.y) - t.z))


Note that mileage is limited because the rewriter cannot change type names; it can only change the names of `ParameterExpression` nodes.

### Constant hoisting

In services like Reaqtor that host and evaluate millions of expressions, it's often useful to detect expressions that are identical modulo constants. These constants may get introduced due to partial evaluation at the client side, e.g. due to closures. An example is shown below:

```csharp
int a = 41;

IQueryable<int> query = from x in xs where x > a select x + 1;

foreach (var x in query)
{
    // use results
}
```

This query will effectively get translated to the following if closures are being eliminates (e.g. using the `FuncletExpression` approach described earlier):

```csharp
IQueryable<int> query = from x in xs where x > 42 select x + 1;
```

Other invocations of the same code may use a different value for `a` and thus generate other queries that are identical modulo this single constant:

```csharp
IQueryable<int> query1 = from x in xs where x > 41 select x + 1;
IQueryable<int> query2 = from x in xs where x > 42 select x + 1;
IQueryable<int> query3 = from x in xs where x > 43 select x + 1;
```

If a high-density service ends up compiling all of these query expressions in order to evaluate them, we can end up with a lot of JITted code on the heap, and pay the price for emitting the code in the first place (e.g. using `System.Reflection.Emit` under the hood). By using the technique of constant hoisting, we can lift constants out of these queries, like this:

```csharp
Func<int, int, IQueryable<int>> query = (c1, c2) => from x in xs where x > c1 select x + c2;

IQueryable<int> query1 = query(41, 1);
IQueryable<int> query2 = query(42, 1);
IQueryable<int> query3 = query(43, 1);
```

The `ConstantHoister` in this library provides support to do this, which is shown below. First, let's write a few user queries that are identical modulo constants.

In [ ]:
Expression<Func<IEnumerable<int>>> query1 = () => Enumerable.Range(1, 10).Where(x => x % 2 == 0).Select(x => x + 9);
Expression<Func<IEnumerable<int>>> query2 = () => Enumerable.Range(2, 20).Where(y => y % 3 == 1).Select(y => y + 8);
Expression<Func<IEnumerable<int>>> query3 = () => Enumerable.Range(3, 30).Where(z => z % 4 == 2).Select(z => z + 7);

Expression expr1 = query1.Body;
Expression expr2 = query2.Body;
Expression expr3 = query3.Body;

Console.WriteLine(expr1);
Console.WriteLine(expr2);
Console.WriteLine(expr3);

Range(1, 10).Where(x => ((x % 2) == 0)).Select(x => (x + 9))


Range(2, 20).Where(y => ((y % 3) == 1)).Select(y => (y + 8))


Range(3, 30).Where(z => ((z % 4) == 2)).Select(z => (z + 7))


To make our setup a little more challenging, note that we've also used different variable names in the three expressions. Modulo the constants, all the queries have the same semantics.

Next, let's explore the `ConstantHoister` facility to hoist out constants for these three query expressions.

In [ ]:
var hoister = new ConstantHoister();

ExpressionWithEnvironment hoisted1 = hoister.Hoist(expr1);

The call to `Hoist` returns an object that contains an expression and an environment. Let's first look at the `Expression` property.

In [ ]:
Console.WriteLine(hoisted1.Expression);

Range(@p0, @p1).Where(x => ((x % @p2) == @p3)).Select(x => (x + @p4))


Note that all of the `ConstantExpression` nodes have been substituted by `ParameterExpression` nodes. In other words, we end up with a parameterized query. Now, let's look at the `Environment`.

In [ ]:
var sb = new StringBuilder();

foreach (var (key, value) in hoisted1.Environment)
{
    sb.AppendLine($"  {key} -> {value}");
}

Console.WriteLine(sb);

  @p0 -> 1
  @p1 -> 10
  @p2 -> 2
  @p3 -> 0
  @p4 -> 9



We can now combine the environment and the hoisted expression to create a parameterized query.

In [ ]:
var constantHoistedExpr = Expression.Lambda(hoisted1.Expression, hoisted1.Environment.Select(kv => kv.Key));

Console.WriteLine(constantHoistedExpr);

(@p0, @p1, @p2, @p3, @p4) => Range(@p0, @p1).Where(x => ((x % @p2) == @p3)).Select(x => (x + @p4))


The original user expression is now equivalent to invoking this query with the values in the environment, like this:

In [ ]:
var reconstructedQueryExpr = Expression.Invoke(constantHoistedExpr, hoisted1.Environment.Select(kv => Expression.Constant(kv.Value, kv.Key.Type)));

Console.WriteLine(reconstructedQueryExpr);

Invoke((@p0, @p1, @p2, @p3, @p4) => Range(@p0, @p1).Where(x => ((x % @p2) == @p3)).Select(x => (x + @p4)), 1, 10, 2, 0, 9)


However, we don't just want to use the `ConstantHoister` to hoist out constants just to put them back using lambda lifting, invocation, and beta reduction. Instead, we want to look up a compiled query expression from the lambda lifted query expression. To do so, let's start off by defining a compiled delegate cache as a dictionary that maps from `LambdaExpression` to `Delegate`.

In [ ]:
var compiledQueries = new Dictionary<LambdaExpression, Delegate>(new ExpressionEqualityComparer());

By using the `ExpressionEqualityComparer`, expressions get compared using value equality semantics rather than reference equality. Note that this will also take care of query expressions being different modulo parameter names in e.g. query clauses.

Next, let's encapsulate our constant hoisting experiments in a single `Evaluate` function that will perform constant hoisting and lambda lifting prior to looking up whether the query has been received before through our `compiledQueries` cache.

In [ ]:
void Evaluate(Expression expression)
{
    var hoister = new ConstantHoister();

    ExpressionWithEnvironment hoisted = hoister.Hoist(expression);

    var constantHoistedExpr = Expression.Lambda(hoisted.Expression, hoisted.Environment.Select(kv => kv.Key));

    if (!compiledQueries.TryGetValue(constantHoistedExpr, out var compiled))
    {
        Console.WriteLine($"Compiling {constantHoistedExpr}");

        compiled = constantHoistedExpr.Compile();
        compiledQueries.Add(constantHoistedExpr, compiled);
    }

    var constants = hoisted.Environment.Select(kv => Expression.Constant(kv.Value, kv.Key.Type)).ToArray();

    var bound = Expression.Invoke(Expression.Constant(compiled, constantHoistedExpr.Type), constants);
    object res = bound.Evaluate();

    Console.WriteLine($"Evaluating {bound} = {res}");
}

Now we can apply `Evaluate` to our query expressions which only differ in the constants used.

In [ ]:
Evaluate(expr1);
Evaluate(expr2);
Evaluate(expr3);

Compiling (@p0, @p1, @p2, @p3, @p4) => Range(@p0, @p1).Where(x => ((x % @p2) == @p3)).Select(x => (x + @p4))


Evaluating Invoke(value(System.Func`6[System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Collections.Generic.IEnumerable`1[System.Int32]]), 1, 10, 2, 0, 9) = System.Linq.Enumerable+WhereSelectEnumerableIterator`2[System.Int32,System.Int32]


Evaluating Invoke(value(System.Func`6[System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Collections.Generic.IEnumerable`1[System.Int32]]), 2, 20, 3, 1, 8) = System.Linq.Enumerable+WhereSelectEnumerableIterator`2[System.Int32,System.Int32]


Evaluating Invoke(value(System.Func`6[System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Collections.Generic.IEnumerable`1[System.Int32]]), 3, 30, 4, 2, 7) = System.Linq.Enumerable+WhereSelectEnumerableIterator`2[System.Int32,System.Int32]


Note we only compiled the query expression once and then went ahead to evaluate the compiled expression three times, with different constants.

More advanced facilities in the `ConstantHoister` enable excluding certain constants. For example, strings passed to the format parameter of `String.Format(string, object[])` or the regular expression parameter of `Regex.Match(string, string)` tend to not vary; in fact, these are strings that encode a small embedded programming language. To exclude such constants, exclusion patterns can be added to the `ConstantHoister` constructor.

In [ ]:
using System.Text.RegularExpressions;

var betterHoister = new ConstantHoister(useDefaultForNull: true, exclusions: new LambdaExpression[]
{
    (Expression<Func<string, string>>)(s => string.Format(s, default(object))),
    (Expression<Func<string, string>>)(s => string.Format(s, default(object), default(object))),
    (Expression<Func<string, string>>)(s => string.Format(s, default(object), default(object), default(object))),
    (Expression<Func<string, string>>)(s => string.Format(s, default(object[]))),
    (Expression<Func<string, Match>>)(pattern => Regex.Match("", pattern)),
});

The patterns in the exclusion list are lambda expressions where the first parameter indicates the position where a constant node should be ignored for hoisting. For example, if the user wrote `Regex.Match("foobar", "[a-z]*")`, then the first argument `"foobar"` will be hoisted as a constant, but the second argument `"[a-z]*"` won't be hoisted. An example is shown below:

In [ ]:
Expression<Func<Match>> f = () => Regex.Match(string.Format("{0}{1}", "foo", "bar"), "[a-z]*");

ExpressionWithEnvironment res = betterHoister.Hoist(f);

Console.WriteLine(res.Expression);

() => Match(Format("{0}{1}", @p0, @p1), "[a-z]*")


Note that only `"foo"` and `"bar"` are getting hoisted while the format string and regular expression constants are kept in the expression tree.

### Type substitution

The `TypeSubstitutionExpressionVisitor` can be used to retype an expression tree from one set of types to another set of types. This involves specifying rules to rebind members involving the types being substituted. In the context of Reaqtor, type substitution is often used to move a tree from one *type space* to another, e.g. from one set of reactive interfaces to another.

A trivial example using `DateTime` and `DateTimeOffset` is shown below:

In [ ]:
var subst = new TypeSubstitutionExpressionVisitor(new Dictionary<Type, Type>
{
    { typeof(DateTime), typeof(DateTimeOffset) }
});

Expression<Func<DateTime>> tomorrow = () => DateTime.Now.AddDays(1);

Expression rewritten = subst.Apply(tomorrow);

Console.WriteLine(rewritten);

() => DateTimeOffset.Now.AddDays(1)


More advanced use cases can be supported by deriving from `TypeSubstitutionExpressionVisitor` to override various `Resolve` methods that are used to resolve reflection objects as any of the types changes. In the sample above, the substitution of `DateTime` to `DateTimeOffset` triggers lookup for `Now` and `AddDays` on `DateTimeOffset`, which can be found successfully. In some cases, more complex mappings are required. An example of a failing rewrite is shown below.

In [ ]:
Expression<Func<DateTime>> birthday = () => new DateTime(1983, 2, 11);

subst.Apply(birthday);

System.InvalidOperationException: No suitable constructor on declaring type 'System.DateTimeOffset' with parameter types (System.Int32, System.Int32, System.Int32) found.
   at System.Linq.CompilerServices.TypeSubstitutionExpressionVisitor.FailResolveConstructor(ConstructorInfo originalConstructor, Type declaringType, Type[] parameters) in D:\Projects\Reaqtive\reaqtor\Nuqleon\Core\LINQ\Nuqleon.Linq.CompilerServices\Expressions\Rewriters\TypeSystem\TypeSubstitutionExpressionVisitor.cs:line 434
   at System.Linq.CompilerServices.TypeSubstitutionExpressionVisitor.ResolveConstructor(ConstructorInfo originalConstructor, Type declaringType, Type[] parameters) in D:\Projects\Reaqtive\reaqtor\Nuqleon\Core\LINQ\Nuqleon.Linq.CompilerServices\Expressions\Rewriters\TypeSystem\TypeSubstitutionExpressionVisitor.cs:line 416
   at System.Linq.CompilerServices.TypeSubstitutionExpressionVisitor.VisitConstructor(ConstructorInfo constructor) in D:\Projects\Reaqtive\reaqtor\Nuqleon\Core\LINQ\Nuqleon.Linq.C

This fails because `DateTimeOffset` does not define a constructor that's isomporhic to the constructor on `DateTime`. To support this type of rewrite, we can override `VisitNew` to handle the case manually. Because `TypeSubstitutionExpressionVisitor` is just a specialized visitor, we can override any of the `Visit` methods if we have to. In this case, that's our only hope because we can't use `ResolveConstructor` to find a `ConstructorInfo` on `DateTimeOffset` that takes three `int` parameters.

In [ ]:
class DateTimeSubstitutor : TypeSubstitutionExpressionVisitor
{
    private static readonly Dictionary<Type, Type> s_map = new Dictionary<Type, Type>
    {
        { typeof(DateTime), typeof(DateTimeOffset) }
    };

    private static readonly ConstructorInfo s_ctorDateTime_IntIntInt = typeof(DateTime).GetConstructor(new[] { typeof(int), typeof(int), typeof(int) });
    private static readonly ConstructorInfo s_ctorDateTimeOffset_DateTime = typeof(DateTimeOffset).GetConstructor(new[] { typeof(DateTime) });

    public DateTimeSubstitutor()
        : base(s_map)
    {
    }

    protected override Expression VisitNew(NewExpression node)
    {
        if (node.Constructor?.DeclaringType == typeof(DateTime))
        {
            if (node.Constructor == s_ctorDateTime_IntIntInt)
            {
                var args = Visit(node.Arguments);

                return Expression.New(s_ctorDateTimeOffset_DateTime, node.Update(args));
            }
        }

        return base.VisitNew(node);
    }
}

Now, we can perform the rewrite.

In [ ]:
var betterSubst = new DateTimeSubstitutor();

Console.WriteLine(betterSubst.Apply(birthday));

() => new DateTimeOffset(new DateTime(1983, 2, 11))


Furthermore, if an expression tree contains a `ConstantExpression` of a type that has to be changed, a call to a `ConvertConstant` virtual method is made, which enables for the insertion of custom conversion logic.

In [ ]:
var now = Expression.Constant(DateTime.Now);

Console.WriteLine(betterSubst.Apply(now));

System.InvalidOperationException: No suitable conversion of constant '02/28/2021 11:14:43' to type 'System.DateTimeOffset' found.
   at System.Linq.CompilerServices.TypeSubstitutionExpressionVisitor.ConvertConstant(Object originalValue, Type newType) in D:\Projects\Reaqtive\reaqtor\Nuqleon\Core\LINQ\Nuqleon.Linq.CompilerServices\Expressions\Rewriters\TypeSystem\TypeSubstitutionExpressionVisitor.cs:line 384
   at System.Linq.CompilerServices.TypeSubstitutionExpressionVisitor.MakeConstant(Object value, Type type) in D:\Projects\Reaqtive\reaqtor\Nuqleon\Core\LINQ\Nuqleon.Linq.CompilerServices\Expressions\Rewriters\TypeSystem\TypeSubstitutionExpressionVisitor.cs:line 673
   at System.Linq.CompilerServices.ExpressionVisitorWithReflection.VisitConstant(ConstantExpression node) in D:\Projects\Reaqtive\reaqtor\Nuqleon\Core\LINQ\Nuqleon.Linq.CompilerServices\Expressions\Visitors\ExpressionVisitorWithReflection.cs:line 263
   at System.Linq.CompilerServices.TypeSubstitutionExpressionVisitor.Visi

To support this case, let's derive from our previous attempt (to reduce code in this notebook), and override `ConvertConstant` as well.

In [ ]:
class EvenBetterDateTimeSubstitutor : DateTimeSubstitutor
{
    protected override object ConvertConstant(object originalValue, Type newType)
    {
        if (newType == typeof(DateTimeOffset) && originalValue?.GetType() == typeof(DateTime))
        {
            return Expression.Convert(Expression.Constant(originalValue, typeof(DateTime)), newType).Evaluate();
        }

        return base.ConvertConstant(originalValue, newType);
    }
}

Finally, we should be able to also convert a `ConstantExpression` from `DateTime` to `DateTimeOffset`.

In [ ]:
var evenBetterSubst = new EvenBetterDateTimeSubstitutor();

Console.WriteLine(evenBetterSubst.Apply(now));

2/28/2021 11:14:43 AM -08:00


### Tupletization

One of the drawbacks of delegate types in .NET is the lack of support for variadic generics, causing us to end up with a whole ladder of types to support functions with different numbers of parameters (cf. `Action<...>` and `Func<...>` families).

While expression trees support the creation of `LambdaExpression` nodes with any number of parameters, it relies on runtime code generation (using `System.Reflection.Emit`) to manufacture delegate types of arities that exceed the available `Func<>` and `Action<>` types. This results in types that are tricky to carry along across machine boundaries (though Bonsai supports ways to represent function types as first-class citizens).

> **Note:** One can argue that functions with more than 16 parameters should be avoided, but keep in mind that expressions are often machine generated. For example, when performing constant hoisting, the number of parameters generated is linear in the number of constants that occurred in an expression.

Let's first have a look at the runtime generation of delegate types by the `Lambda` factory.

In [ ]:
for (int i = 0; i < 20; i++)
{
    Console.WriteLine(Expression.Lambda(Expression.Empty(), Enumerable.Range(1, i).Select(j => Expression.Parameter(typeof(int)))).Type);
}

for (int i = 0; i < 20; i++)
{
    Console.WriteLine(Expression.Lambda(Expression.Constant(42), Enumerable.Range(1, i).Select(j => Expression.Parameter(typeof(int)))).Type);
}

System.Action


System.Action`1[System.Int32]


System.Action`2[System.Int32,System.Int32]


System.Action`3[System.Int32,System.Int32,System.Int32]


System.Action`4[System.Int32,System.Int32,System.Int32,System.Int32]


...

System.Func`16[System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32]


System.Func`17[System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32]


Delegate18$4


Delegate19$5


Delegate20$6


This library provides a mechanism to normalize lambda expressions to unary functions using a technique called tupletization. Let's first construct a couple of sample expression trees below:

In [ ]:
var xs = Enumerable.Range(1, 20).Select(i => Expression.Parameter(typeof(int), "x" + i)).ToArray();
var sum = xs.Cast<Expression>().Aggregate((l, r) => Expression.Add(l, r));

Expression<Func<int>> f1 = () => 42;
Expression<Func<int, int>> f2 = x => x + 1;
Expression<Func<int, int, int>> f3 = (x, y) => x * y;
LambdaExpression f20 = Expression.Lambda(sum, xs);

Expression<Action> a1 = () => Console.WriteLine(42);
Expression<Action<int>> a2 = x => Console.WriteLine(x + 1);
Expression<Action<int, int>> a3 = (x, y) => Console.WriteLine(x * y);
LambdaExpression a20 = Expression.Lambda(Expression.Call(typeof(Console).GetMethod(nameof(Console.WriteLine), new[] { typeof(int) }), sum), xs);

var exprs = new LambdaExpression[]
{
    f1,
    f2,
    f3,
    f20,
    a1,
    a2,
    a3,
    a20
};

Using the `ExpressionTupletizer` we can convert back and forth between the N-ary function form and a so-called tupletized form, using `Pack` and `Unpack` methods. We'll illustrate this roundtripping in the next cell.

In [ ]:
foreach (var expr in exprs)
{
    var sb = new StringBuilder();
    sb.AppendLine(expr.ToString());

    LambdaExpression packed = ExpressionTupletizer.Pack(expr, voidType: typeof(ValueTuple));
    sb.AppendLine("  Pack:   " + packed.ToString());
    
    LambdaExpression unpacked = ExpressionTupletizer.Unpack(packed, voidType: typeof(ValueTuple));
    sb.AppendLine("  Unpack: " + unpacked.ToString());

    Console.WriteLine(sb);
}

() => 42
  Pack:   _ => 42
  Unpack: () => 42



x => (x + 1)
  Pack:   t => (t.Item1 + 1)
  Unpack: p0 => (p0 + 1)



(x, y) => (x * y)
  Pack:   t => (t.Item1 * t.Item2)
  Unpack: (p0, p1) => (p0 * p1)



(x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20) => (((((((((((((((((((x1 + x2) + x3) + x4) + x5) + x6) + x7) + x8) + x9) + x10) + x11) + x12) + x13) + x14) + x15) + x16) + x17) + x18) + x19) + x20)
  Pack:   t => (((((((((((((((((((t.Item1 + t.Item2) + t.Item3) + t.Item4) + t.Item5) + t.Item6) + t.Item7) + t.Rest.Item1) + t.Rest.Item2) + t.Rest.Item3) + t.Rest.Item4) + t.Rest.Item5) + t.Rest.Item6) + t.Rest.Item7) + t.Rest.Rest.Item1) + t.Rest.Rest.Item2) + t.Rest.Rest.Item3) + t.Rest.Rest.Item4) + t.Rest.Rest.Item5) + t.Rest.Rest.Item6)
  Unpack: (p0, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14, p15, p16, p17, p18, p19) => (((((((((((((((((((p0 + p1) + p2) + p3) + p4) + p5) + p6) + p7) + p8) + p9) + p10) + p11) + p12) + p13) + p14) + p15) + p16) + p17) + p18) + p19)



() => WriteLine(42)
  Pack:   _ => WriteLine(42)
  Unpack: () => WriteLine(42)



x => WriteLine((x + 1))
  Pack:   t => WriteLine((t.Item1 + 1))
  Unpack: p0 => WriteLine((p0 + 1))



(x, y) => WriteLine((x * y))
  Pack:   t => WriteLine((t.Item1 * t.Item2))
  Unpack: (p0, p1) => WriteLine((p0 * p1))



(x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20) => WriteLine((((((((((((((((((((x1 + x2) + x3) + x4) + x5) + x6) + x7) + x8) + x9) + x10) + x11) + x12) + x13) + x14) + x15) + x16) + x17) + x18) + x19) + x20))
  Pack:   t => WriteLine((((((((((((((((((((t.Item1 + t.Item2) + t.Item3) + t.Item4) + t.Item5) + t.Item6) + t.Item7) + t.Rest.Item1) + t.Rest.Item2) + t.Rest.Item3) + t.Rest.Item4) + t.Rest.Item5) + t.Rest.Item6) + t.Rest.Item7) + t.Rest.Rest.Item1) + t.Rest.Rest.Item2) + t.Rest.Rest.Item3) + t.Rest.Rest.Item4) + t.Rest.Rest.Item5) + t.Rest.Rest.Item6))
  Unpack: (p0, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14, p15, p16, p17, p18, p19) => WriteLine((((((((((((((((((((p0 + p1) + p2) + p3) + p4) + p5) + p6) + p7) + p8) + p9) + p10) + p11) + p12) + p13) + p14) + p15) + p16) + p17) + p18) + p19))



In here, the use of `voidType` is necessary to specify how a lambda expression without any parameters is turned into a unary lambda expression with one parameter. To do so, the introduced dummy parameter needs to have a type, which is represented using `voidType`. The use of `ValueTuple` is well-suited for this purpose.

Reaqtor uses the tupletized form in quite a few places to standardize on unary functions at the base layers of the platform. Any function arity is supported at layers higher up, which are merely syntactic sugar to construct expressions in tuple form.

### `AnonynmousTypeTupletizer`

Another rewriting utility that relies on tuples is `AnonynmousTypeTupletizer`. This expression rewriter is a type substitution visitor that replaces anonymous types (e.g. produced by `let` clauses in query expression, due to the use of transparent identifiers) by tuple types. This can be useful to shake off compiler-generated types prior to carrying out further rewrite steps (or prior to serialization of the expression for shipment to another machine). For example:

In [ ]:
var query = from x in new[] { 1, 2, 3 }.AsQueryable()
            let y = x + 1
            select x + y;

var queryExpr = CompilerGeneratedNameEliminator.Prettify(query.Expression);

Console.WriteLine(queryExpr);

System.Int32[].Select(x => new <>f__AnonymousType0#51`2(x = x, y = (x + 1))).Select(t => (t.x + t.y))


Rather than allocating anonymous types for the transparent identifiers, we can replace these by tuple types instead. This is achieved through `AnonymousTypeTupletizer.Tupletize`, which takes up to three parameters. Besides the expression to rewrite, the `unitValue` specifies the expression to use when replacing `new {}` anonymous types (due to the lack of a non-generic `Tuple` type in the BCL), and `excludeVisibleTypes` prevents the tuple rewriting from changing the type of the top-level expression. We'll get to this detail in a bit, but let's first apply the rewrite to our query expression.

In [ ]:
var tupletizedQueryExpr = AnonymousTypeTupletizer.Tupletize(queryExpr, unitValue: Expression.Constant(new object()), excludeVisibleTypes: true);

Console.WriteLine(tupletizedQueryExpr);

System.Int32[].Select(x => new Tuple`2(x, (x + 1))).Select(t => (t.Item1 + t.Item2))


In the resulting expression, we have no anonymous types left. This simplifies the expression by reducing the number of types it relies on. This in turn makes tasks like expression serialization easier.

### CPS transformations

This library provides very rudimentary support for continuation passing style (CPS) transforms of code which can be used in the context of binding query execution plans to asynchronous infrastructure.

**Note:** The CPS transform support in this library only works for expressions because it originates from a LINQ provider toolkit we wrote in the .NET 3.5 days, prior to support for statement nodes. A full-blown CPS transformation framework existed in the context of Volta's tier splitting work, but it was written in CCI (the Common Compiler Infrastructure) and never got ported to .NET 4.0's expressions. While Nuqleon doesn't use this implementation of CPS transform directly (anymore), we're keeping it around for other uses (e.g. execution plans in some internal stores). A more complete CPS transformation framework could be built, especially in conjunction with the work on modernized expression trees with support for async lambdas and `await` expressions.

CPS transforms involve rewriting method invocations such as `Add(a, b)` to `Add(a, b, ret => ...)` where the result is provided through a callback rather than a returned result. This then enables the use of asynchronous functions.

> **Note:** This functionality has been used for execution plans in stores that have asynchronous callback-based `Get`, `Read`, `Enumerate`, etc. operations, but the user's code uses synchronous operations instead. By enabling rewrites to synchronous stub methods (e.g. `T Get<T>(string key)`) and annotating these stubs with a `UseAsyncMethod` attribute, the CPS transformation framework performs a set of tree rewrites that replace the stub method calls with their async counterparts (e.g. `void Get<T>(string key, Action<T>)`).

To illustrate this facility, let's first build the most trivial CPS transformation demo with a static `Add` method.

In [ ]:
static class Calculator
{
    [UseAsyncMethod]
    public static int Add(int x, int y) => throw new NotImplementedException();

    public static void Add(int x, int y, Action<int> ret) => ret(x + y);
}

Given this definition, a user can write an expression of the form `Add(1, Add(2, 3))` but not that the method does not have an implementation. Instead, we want to translate the user's code to a corresponding asynchronous implementation, which is indicated by the `[UseAsyncMethod]` annotation.

In [ ]:
Expression<Func<int>> calcExpr = () => Calculator.Add(1, Calculator.Add(2, 3));

Console.WriteLine(calcExpr);

() => Add(1, Add(2, 3))


If the underlying evaluation infrastructure happens to be asynchronous, we can use the CPS rewriter to retarget the call to `int Add(int, int)` to `void Add(int, int, Action<int>)` which is the calback-based equivalent operation. In the code below, we perform this rewrite and specify the continuation we want to invoke upon obtaining the result of evaluating the expression asynchronously.

In [ ]:
var rewriter = new ClassicCpsRewriter();

Expression<Action<int>> continuation = ret => Console.WriteLine(ret);

Expression asyncExpr = rewriter.Rewrite(calcExpr, continuation);

Console.WriteLine(asyncExpr);

Add(2, 3, x0 => Add(1, x0, ret => WriteLine(ret)))


We can also evaluate the resulting expression.

In [ ]:
Expression.Lambda<Action>(asyncExpr).Compile()();

6


A more realistic example of using the CPS transformation is to map an execution plan onto asynchronous implementations of primitive operations, e.g. in the context of a database product. Note that the CPS utilities in this library predate the introduction of `async` and `await`, but it is possible to combine these facilities with `Task<T>` based APIs. An example is shown below.

In [ ]:
static class AsyncCalculator
{
    [UseAsyncMethod]
    public static int Add(int x, int y) => throw new NotImplementedException();

    public static async void Add(int x, int y, Action<int> ret)
    {
        await Task.Delay(1000);
        ret(x + y);
    }
}

Once more, we create a sample expression.

In [ ]:
Expression<Func<int>> calcExpr = () => AsyncCalculator.Add(1, AsyncCalculator.Add(2, 3));

Console.WriteLine(calcExpr);

() => Add(1, Add(2, 3))


In order to make the entire CPS-transformed expression return a `Task<int>`, we need to make the final continuation use a `TaskCompletionSource<int>`. To do so, we can build an expression tree that's very similar to the machinery of `async` methods in C# 5.0 and beyond. First, we'll craft some variables for `TaskCompletionSource<int>` and an expression for the `Action<int>` continuation.

In [ ]:
var tcs = Expression.Parameter(typeof(TaskCompletionSource<int>), "tcs");

var result = Expression.Parameter(typeof(int), "res");
var setResult = Expression.Call(tcs, typeof(TaskCompletionSource<int>).GetMethod(nameof(TaskCompletionSource<int>.SetResult), new[] { typeof(int) }), result);
var continuation = Expression.Lambda<Action<int>>(setResult, result);

Console.WriteLine(continuation);

res => tcs.SetResult(res)


With a continuation expression available, we can now perform the CPS transformation on `calcExpr`.

In [ ]:
var rewriter = new ClassicCpsRewriter();

Expression asyncExpr = rewriter.Rewrite(calcExpr, continuation);

Console.WriteLine(asyncExpr);

Add(2, 3, x0 => Add(1, x0, res => tcs.SetResult(res)))


Finally, we wrap the whole fanfare into a async ramp function that instantiates the `TaskCompletionSource<int>`, stores it in a local, kicks off the async operation, and finally returns the `Task`.

In [ ]:
var task = Expression.Property(tcs, typeof(TaskCompletionSource<int>).GetProperty(nameof(TaskCompletionSource<int>.Task)));
var newTcs = Expression.New(typeof(TaskCompletionSource<int>).GetConstructor(Type.EmptyTypes));

var calcAsync =
    Expression.Lambda<Func<Task<int>>>(
        Expression.Block(
            new[] { tcs },
            Expression.Assign(tcs, newTcs),
            asyncExpr,
            task
        )
    );

Console.WriteLine(calcAsync);

() => {var tcs; ... }


At long last, we can evaluate the expression and wait for the task's completion (here using `.Result` for simplicity in the notebook).

In [ ]:
Task<int> t = calcAsync.Evaluate<Task<int>>();

Console.WriteLine(t.Result); // This should take 2 seconds to complete due to the Task.Delay(1000) inside Add, which gets invoked twice.

6


To support propagation of errors, the `ClassicCpsRewriterWithErrorPropagation` rewriter can be used instead. Rather than rewriting synchronous methods to methods taking in a single callback of type `Action<T>`, an additional callback of type `Action<Exception>` is passed for the error continuation. We can illustrate this using a `Div` function that can throw `DivideByZeroException`.

In [ ]:
static class AsyncFancyCalculator
{
    [UseAsyncMethod]
    public static int Div(int x, int y) => throw new NotImplementedException();

    public static async void Div(int x, int y, Action<int> ret, Action<Exception> error)
    {
        await Task.Delay(1000);
        if (y == 0)
        {
            error(new DivideByZeroException());
        }
        else
        {
            ret(x / y);
        }
    }
}

We'll repeat all of the steps carried out earlier, but this time using a `ClassicCpsRewriterWithErrorPropagation` instead.

In [ ]:
static Expression<Func<Task<int>>> MakeAsync(Expression<Func<int>> expr)
{
    // TaskCompletionSource<int> tcs
    var tcs = Expression.Parameter(typeof(TaskCompletionSource<int>), "tcs");

    // int res
    var result = Expression.Parameter(typeof(int), "res");

    // Exception ex
    var exception = Expression.Parameter(typeof(Exception), "ex");

    // tcs.SetResult(res)
    var setResult = Expression.Call(tcs, typeof(TaskCompletionSource<int>).GetMethod(nameof(TaskCompletionSource<int>.SetResult), new[] { typeof(int) }), result);

    // tcs.SetException(ex)
    var setException = Expression.Call(tcs, typeof(TaskCompletionSource<int>).GetMethod(nameof(TaskCompletionSource<int>.SetException), new[] { typeof(Exception) }), exception);

    // (int res) => tcs.SetResult(res)
    var onSuccess = Expression.Lambda<Action<int>>(setResult, result);

    // (Exception ex) => tcs.SetException(ex)
    var onError = Expression.Lambda<Action<Exception>>(setException, exception);

    // Op(arg_1, ..., arg_n, (int res) => tcs.SetResult(res), (Exception ex) => tcs.SetException(ex))
    var asyncExpr = new ClassicCpsRewriterWithErrorPropagation().Rewrite(expr, onSuccess, onError);

    // new TaskCompletionSource<int>()
    var newTcs = Expression.New(typeof(TaskCompletionSource<int>).GetConstructor(Type.EmptyTypes));

    // tcs.Task
    var task = Expression.Property(tcs, typeof(TaskCompletionSource<int>).GetProperty(nameof(TaskCompletionSource<int>.Task)));

    // new Func<Task<int>>(() =>
    // {
    //     var tcs = new TaskCompletionSource<int>();
    //     Op(arg_1, ..., arg_n, (int res) => tcs.SetResult(res), (Exception ex) => tcs.SetException(ex))
    //     return tcs.Task;
    // })
    var calcAsync =
        Expression.Lambda<Func<Task<int>>>(
            Expression.Block(
                new[] { tcs },
                Expression.Assign(tcs, newTcs),
                asyncExpr,
                task
            )
        );

    return calcAsync;
}

Let's now run this for two different expressions and observe the result or error getting propagated through the `Task<int>` object.

In [ ]:
Task<int> resSuccess = MakeAsync(() => AsyncFancyCalculator.Div(3, 2)).Compile()();
Task<int> resError = MakeAsync(() => AsyncFancyCalculator.Div(1, 0)).Compile()();

Console.WriteLine(await resSuccess);
Console.WriteLine(await resError); // Will throw!

1


System.DivideByZeroException: Attempted to divide by zero.
   at Submission#93.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

## Expression tree evaluation

Evaluation of expression trees can be done using `Compile` methods on `LambdaExpression` and `Expression<TDelegate>` as shown below:

In [ ]:
Expression<Func<int>> f = () => 42;

Func<int> d = f.Compile();

int x = d();

Console.WriteLine(x);

42


By default, `Compile` will use `System.Reflection.Emit` to generate IL code at runtime. There's also support for using an interpreter using a Boolean `preferInterpretation` flag, which can be faster if the expression is simple. Furthermore, it can reduce costs (due to IL code generation and JIT compilation) if the expression is only to be evaluated once. If the expression is evaluated often or is rather complex, compiled expressions tend to perform better.

In [ ]:
Func<int> d = f.Compile(preferInterpretation: true);

int x = d();

Console.WriteLine(x);

42


All of the above are part of the `System.Linq.Expressions` library in the BCL. This library adds some functionality on top of this.

### The `Evaluate` and `Funcletize` extension methods

First of all, this library provides a number of methods that make it easier to perform evaluation of any `Expression` without having to manually construct a `LambdaExpression` or `Expression<TDelegate>` around it. This wrapping step is simplified using `Funcletize`, as shown below:

In [ ]:
Expression expr = Expression.Constant(42);

Expression<Func<int>> eval = expr.Funcletize<int>();

Console.WriteLine(eval);

() => 42


This can then be used to call `Compile()` in order to perform evaluation. However, if the end goal is to just do that, we can directly call `Evaluate` instead, as shown below:

In [ ]:
int answer = expr.Evaluate<int>();

Console.WriteLine(answer);

42


Where things become more interesting is in the additional `ICompiledDelegateCache` that can be passed to `Evaluate` overloads.

### Using compiled delegate caches with `ICompiledDelegateCache`

Compiled delegate caches use expression tree equality comparison to avoid reocmpiling expression trees that have been compiled previously. We demonstrated this technique earlier in this notebook by building a `Dictionary<LambdaExpression, Delegate>`, when discussing constant hoisting. The built-in support for `ICompiledDelegateCache` makes this task easier, while also providing for caching policies. To illustrate the effect of compiled delegate caching, we'll first define a little benchmark utility.

In [ ]:
using System.Time;

class MemoryClock : IClock
{
    public long Now => GC.GetAllocatedBytesForCurrentThread();
}

IStopwatch swMem = StopwatchFactory.FromClock(new MemoryClock()).Create();
IStopwatch swTime = StopwatchFactory.Diagnostics.Create();

void Benchmark(string title, Action test, int n)
{
    swMem.Restart();
    swTime.Restart();

    for (int i = 0; i < n; i++)
    {
        test();
    }

    swTime.Stop();
    swMem.Stop();

    Console.WriteLine($"{title} completed in {swTime.ElapsedMilliseconds} ms and allocated {swMem.ElapsedTicks} bytes.");
}

Next, we'll craft a little expression tree to test compilation with. Note we won't evaluate this expression; instead, we'll use `ICompiledDelegateCache` as a parameter to a `Compile` extension method defined in this library. Our goal is to compare the cost of always calling `Compile()` versus calling `Compile(ICompiledDelegateCache)`.

In [ ]:
Expression<Func<string, int>> expr = s => s.ToUpper().Substring(1, 2).ToLower().Length - 1;

To test compiled delegate caches, we'll start off with a `SimpleCompiledDelegateCache` which has an unbounded cache size. Alternatively we could use the `LeastRecentlyUsedCompiledDelegateCache` which has an LRU cache eviction policy.

In [ ]:
var cache = new SimpleCompiledDelegateCache();

Benchmark("Without caching", () => expr.Compile(), 10_000);
Benchmark("With caching", () => expr.Compile(cache), 10_000);

Without caching completed in 1677 ms and allocated 49214568 bytes.


With caching completed in 246 ms and allocated 47700904 bytes.


The time taken to compile an expression that has been compiled before is significantly shorter because we're avoiding going into `System.Reflection.Emit` and (down the line) triggering JIT compilation. In the code below, we'll also invoke the compiled delegate.

In [ ]:
var cache = new SimpleCompiledDelegateCache();

Benchmark("Without caching", () => expr.Compile()("foobar"), 10_000);
Benchmark("With caching", () => expr.Compile(cache)("foobar"), 10_000);

Without caching completed in 1881 ms and allocated 49280624 bytes.


With caching completed in 206 ms and allocated 48731968 bytes.


The difference is even more pronounced when we try to compile expressions concurrently, which mimics exactly what happens during recovery of query evaluators in Reaqtor. Due to locks inside the CLR (around the code generation area), we end up with a slow down due to lock contention. It should also be noted that the total memory cost reported here is far lower than the real cost involved due to much of the memory allocations happening on the native side of the CLR. Our `MemoryClock` only accounts for managed memory allocations.

In [ ]:
using System.Threading;

void BenchmarkConcurrent(string title, Action compileAndEval, int n)
{
    long totalMs = 0L;
    long totalBytes = 0L;

    var threads =
        Enumerable.Range(0, Environment.ProcessorCount).Select(_ =>
            new Thread(() =>
            {
                IStopwatch swMem = StopwatchFactory.FromClock(new MemoryClock()).StartNew();
                IStopwatch swTime = StopwatchFactory.Diagnostics.StartNew();

                for (int i = 0; i < n; i++)
                {
                    compileAndEval();
                }

                Interlocked.Add(ref totalMs, swTime.ElapsedMilliseconds);
                Interlocked.Add(ref totalBytes, swMem.ElapsedTicks);
            })
        ).ToArray();

    foreach (var thread in threads)
    {
        thread.Start();
    }

    foreach (var thread in threads)
    {
        thread.Join();
    }

    Console.WriteLine($"{title} completed in {totalMs} ms and allocated {totalBytes} bytes (using {Environment.ProcessorCount} threads).");
}

var cache = new SimpleCompiledDelegateCache();

BenchmarkConcurrent("Without caching", () => expr.Compile()("foobar"), 10_000);
BenchmarkConcurrent("With caching", () => expr.Compile(cache)("foobar"), 10_000);

Without caching completed in 470784 ms and allocated 607413072 bytes (using 12 threads).


With caching completed in 6037 ms and allocated 579592976 bytes (using 12 threads).


Note that `SimpleCompiledDelegateCache` is thread-safe, so it's okay to have concurrent compilations all trying to access the cache.

Compilation using compiled delegate caching has a few more knobs that can be turned using overloads of `Compile`. The most flexible overload has the following signature:

```csharp
public static T Compile<T>(this Expression<T> expression, ICompiledDelegateCache cache, bool outliningEnabled, IConstantHoister hoister);
```

Outlining (as the opposite of inlining) can be used when expressions have nested lambdas. For example, `xs => xs.Where(x => x > 0).Select(x => x * 2)` has inner lambdas `x => x > 0` and `x => x * 2`. By using outlining, we'll compile this inner lambda using caching as well, allowing for reuse if an identical lambda occurs elsewhere. For example, if another expression has `ys => ys.Select(y => y + 1).Where(y => y > 4)`, we can reuse a cached compiled lambda of the form `t => t > c` where `c` is a hoisted constant. Both `x => x > 0` and `y => y > 4` match this pattern, for different constants.

A custom hoister can also be specified, for example to add exclusion rules (e.g. for the format string in `string.Format(string, object[])`) as described earlier in this notebook.

We'll finish our discussion of compiled delegate caching by illustrating the use of outlining (which is enabled by default when using simpler overloads of `Compile`).

In [ ]:
Expression<Func<IEnumerable<int>, IEnumerable<int>>> queryExpr1 = xs => xs.Where(x => x > 0).Take(1);
Expression<Func<IEnumerable<int>, IEnumerable<int>>> queryExpr2 = ys => ys.Take(2).Where(y => y > 3);

Both expressions have a nested lambda of the form `t => t > c` for some constant value `c`. When outlining is enabled, we'll end up hoisting these inner lambdas out of the original expression, and will compile (and cache) them separately. We can visualize this by making use of events declared on `LeastRecentlyUsedCompiledDelegateCache` which can be used to inspect additions, hits, and removals of entries in the cache.

In [ ]:
var cache = new LeastRecentlyUsedCompiledDelegateCache(capacity: 16);

cache.Added += (object o, CacheEventArgs args) =>
{
    Console.WriteLine($"Added {args.Lambda} to cache.  Delegate = {args.Delegate.GetHashCode()}");
};

cache.Hit += (object o, CacheEventArgs args) =>
{
    Console.WriteLine($"Retrieved {args.Lambda} from cache. Delegate = {args.Delegate.GetHashCode()}");
};

Now we can try to compile the first query expression using the cache, with outlining enabled (which is the default behavior for the simpler `Compile` overload used below).

In [ ]:
queryExpr1.Compile(cache);

Added t => x => (x > t.Item1) to cache.  Delegate = 1753183328


Added t => xs => xs.Where(t.Item1).Take(t.Item2) to cache.  Delegate = 153197105


Note that two delegates were added to the cache. One for the inner lambda and one for the outer lambda. If we try to compile the same query expression again, we should see two hits.

In [ ]:
queryExpr1.Compile(cache);

Retrieved t => x => (x > t.Item1) from cache. Delegate = 1753183328


Retrieved t => xs => xs.Where(t.Item1).Take(t.Item2) from cache. Delegate = 153197105


Let's finally try to compile the second expression, where the inner lambda is identical modulo parameter names and constant values. We should see a hit for this lambda, but not for the outer lambda which has a different structure than the first expression.

In [ ]:
queryExpr2.Compile(cache);

Retrieved t => y => (y > t.Item1) from cache. Delegate = 1753183328


Added t => ys => ys.Take(t.Item1).Where(t.Item2) to cache.  Delegate = 896984621


## Expression tree optimization

This library provides a few expression tree optimizations. More optimizations are available in `Nuqleon.Linq.Expressions.Optimizers`.

### Delegate invocation inlining

The `DelegateInvocationInliner` provides a very narrow optimization that looks for `InvocationExpression` nodes whose target is a `ConstantExpression` containing a delegate-typed expression. If the delegate has an invocation list with a single invocation target (i.e. it's not a multicast delegate with multiple invocation targets attached to it), the tree is rewritten to a `MethodCallExpression` for the method targeted by the delegate.

This optimization can be useful after carrying out binding steps where the binding targets for functions are `ConstantExpression`s containing delegates pointing at a function's implementation.

As an example, consider an unbound expression `abs(1)`, as crafted below.

In [ ]:
var abs = Expression.Parameter(typeof(Func<int, int>), "abs");
var expr = Expression.Invoke(abs, Expression.Constant(1));

Console.WriteLine(expr.ToCSharpString());

abs(1)


Note we're using `ToCSharpString` which prints the expression tree in a C#-like syntax. We'll discuss this later in the notebook.

Given this unbound expression, we can perform binding steps as we've illustrated before in the context of beta reduction. First, we'll find all of the unbound variables using the `FreeVariableScanner`.

In [ ]:
var variables = FreeVariableScanner.Scan(expr).ToArray();

foreach (var variable in variables)
{
    Console.WriteLine($"{variable.Name} : {variable.Type.ToCSharpString()}");
}

abs : Func<int, int>


Note we're using `ToCSharpString` on an instance of type `Type` to print the type using C#-like syntax. We'll discuss this later in the notebook.

Just like we've done before while discussing beta reduction, we'll go ahead and perform lambda lifting to declare all of the unbound variables in a surrounding lambda expression.

In [ ]:
var lifted = Expression.Lambda(expr, variables);

Console.WriteLine(lifted.ToCSharpString());

(Func<int, int> abs) => abs(1)


As the next step to achieve expression binding, we'll introduce a registry of expressions to bind. This is similar to what we've done for beta reduction, but we'll make a sight change this time around.

In [ ]:
var registry = new Dictionary<string, Expression>
{
    { "abs", Expression.Constant(new Func<int, int>(Math.Abs)) }
};

The difference lies in the expression used to bind `abs`. Rather than using an `Expression<Func<int, int>>` of the form `x => Math.Abs(x)`, we're using a `ConstantExpression` that holds a delegate to `Math.Abs`.

Given this registry, we can bind the functions and construct an `InvocationExpression` around the `LambdaExpression` we built earlier.

In [ ]:
var bindings = variables.Select(variable => registry[variable.Name]).ToArray();

var bound = Expression.Invoke(lifted, bindings);

Console.WriteLine(bound.ToCSharpString());

((Func<Func<int, int>, int>)((Func<int, int> abs) => abs(1)))(__c0)


Note the presence of `__c0` in the textual output, which represents an opaque `ConstantExpression` which could not be printed in C# syntax. This is our delegate to `Math.Abs`. Using `ToCSharp` rather than `ToCSharpExpression`, we can inspect this as well.

In [ ]:
CSharpExpression cs = bound.ToCSharp();

Console.WriteLine($"Global variables: {{ {string.Join(", ", cs.GlobalVariables)} }}");
Console.WriteLine($"Code: {cs.Code}");
Console.WriteLine($"Constants: {string.Join(", ", cs.Constants.Select(kv => $"{kv.Key} : {kv.Value.Value.ToString()}"))}");

Global variables: {  }


Code: ((Func<Func<int, int>, int>)((Func<int, int> abs) => abs(1)))(__c0)


Constants: __c0 : System.Func`2[System.Int32,System.Int32]


To simplify the bound expression, we can apply beta reduction.

In [ ]:
var simplified = BetaReducer.Reduce(bound);

Console.WriteLine(simplified.ToCSharpString());

__c0(1)


The result is an `InvocationExpression` that will invoke the delegate stored in the `ConstantExpression`. This does have some overhead (due to extracting the constant and invoking the delegate) which can be reduced by using `DelegateInvocationInliner` to unpack the delegate and call the underlying method directly using a `MethodCallExpression`.

In [ ]:
var result = DelegateInvocationInliner.Apply(simplified, inlineNonPublicMethods: false);

Console.WriteLine(result.ToCSharpString());

Math.Abs(1)


### Expression tree interning

Because expression trees are immutable, they can be shared safely. For example, if two expressions have a common subexpression, the same object can be used to represent that common subexpression:

```csharp
f() + g() + h()
```

and

```csharp
f() + g() + i()
```

can reuse the common subexpression `f() + g()`. However, it's not always possible to figure out that two or more expressions have commonalities that present opportunities for sharing. For example, when expression visitors are used to manipulate expression trees, commonalities may not occur until rewrites have completed.

As an example, let's build a few simpler expressions that have a opportunities for sharing:


In [ ]:
var expr1 = Expression.Add(Expression.Multiply(Expression.Constant(1), Expression.Constant(2)), Expression.Constant(3));
var expr2 = Expression.Add(Expression.Multiply(Expression.Constant(1), Expression.Constant(2)), Expression.Constant(3));
var expr3 = Expression.Add(Expression.Multiply(Expression.Constant(1), Expression.Constant(2)), Expression.Constant(4));
var expr4 = Expression.Subtract(Expression.Multiply(Expression.Constant(1), Expression.Constant(2)), Expression.Constant(3));

Console.WriteLine(expr1);
Console.WriteLine(expr2);
Console.WriteLine(expr3);
Console.WriteLine(expr4);

((1 * 2) + 3)


((1 * 2) + 3)


((1 * 2) + 4)


((1 * 2) - 3)


Expression tree interning allows for the detection of common subexpressions across a "forest" of trees, and rewrite expressions to allow for reuse of common subexpressions, thus reducing the memory utilitization. This is very similar to `string.Intern`, although the latter only operates on complete strings, while expression tree interning can consider every subexpression.

> **Note:** Interning is quite expensive due to the computations involved to figure out commonalities between expressions. It's only recommended to use interning if expressions are long-lived. In the context of Nuqleon, this is most useful when expression trees are kept alive in registries or on quotations that are needed to support higher-order query operators (e.g. in `xs.SelectMany(x => f(x))`, the expression `x => f(x)` is kept - even after compilation to a delegate - because inner subscriptions need to be able to construct an expression, which will get checkpointed, that represents the observable `f(x)` for a given value of `x`).

Let's try applying interning to the expressions shown above.

In [ ]:
var cache = new ExpressionInterningCache();

var expr1_i = expr1.Intern(cache);
var expr2_i = expr2.Intern(cache);
var expr3_i = expr3.Intern(cache);
var expr4_i = expr4.Intern(cache);

Console.WriteLine(expr1_i);
Console.WriteLine(expr2_i);
Console.WriteLine(expr3_i);
Console.WriteLine(expr4_i);

((1 * 2) + 3)


((1 * 2) + 3)


((1 * 2) + 4)


((1 * 2) - 3)


We can start by spot-checking a few equalities to check whether interning had an effect. For example, `expr1_i` and `expr2_i` should be identical, even though `expr1` and `expr2` were not.

In [ ]:
Console.WriteLine($"(expr1   == expr2  ) = {expr1   == expr2  }");
Console.WriteLine($"(expr1_i == expr2_i) = {expr1_i == expr2_i}");

(expr1   == expr2  ) = False


(expr1_i == expr2_i) = True


In order to investigate interior nodes for reference equality, we can build a simple visitor that will print all nodes and their hash code.

In [ ]:
class PrintHashCodeVisitor : ExpressionVisitor
{
    public override Expression Visit(Expression node)
    {
        if (node != null)
        {
            Console.WriteLine($"{node} [{node.GetHashCode()}]");
        }

        return base.Visit(node);
    }
}

We can first run this visitor on the original expressions to see that all nodes are unique.

In [ ]:
var printer = new PrintHashCodeVisitor();

printer.Visit(expr1);
printer.Visit(expr2);
printer.Visit(expr3);
printer.Visit(expr4);

((1 * 2) + 3) [51702607]


(1 * 2) [21677825]


1 [58579833]


2 [14353293]


3 [31461808]


...

((1 * 2) - 3) [28267618]


(1 * 2) [29122994]


1 [27878069]


2 [66236554]


3 [67049879]


When we run the visitor over the interned nodes, we can see the effect of deduplicating nodes that are equivalent.

In [ ]:
printer.Visit(expr1_i);
printer.Visit(expr2_i);
printer.Visit(expr3_i);
printer.Visit(expr4_i);

((1 * 2) + 3) [51702607]


(1 * 2) [21677825]


1 [58579833]


2 [14353293]


3 [31461808]


...

((1 * 2) - 3) [12841208]


(1 * 2) [21677825]


1 [58579833]


2 [14353293]


3 [31461808]


Expression tree interning only considers semantic equivalence of expressions. For example, interning `x => x + 1` and `y => y + 1` can result in variable renaming because both lambda expressions are equivalent.

> **Note**: If this is not desirable, one can specify a custom `ExpressionEqualityComparator` that overrides the behavior for equality of `ParameterExpression` nodes to include the `Name` property.

In [ ]:
Expression<Func<int, int>> f1 = x => x + 1;
Expression<Func<int, int>> f2 = y => y + 1;

var f1_i = f1.Intern();
var f2_i = f2.Intern();

Console.WriteLine(f1_i);
Console.WriteLine(f2_i);
Console.WriteLine(f1_i == f2_i);

x => (x + 1)


x => (x + 1)


True


## Diagnostics

To assist with logging or debugging of expression trees, this library provides a `ToCSharpString` extension method for `Expression` which produces C#-like syntax to represent the tree. Not all constructs in expression trees can be (correctly) represented in C# (e.g. `LoopExpression` having a result), so the resulting syntax just looks and feels like C#.

> **Note:** Better expression tree printing support is available in https://github.com/bartdesmet/ExpressionFutures/tree/master/CSharpExpressions.

In [ ]:
var expr = Expression.Add(Expression.Constant(1), Expression.Parameter(typeof(int), "x"));

Console.WriteLine(expr.ToCSharpString());

1 + x


An additional Boolean parameter called `allowCompilerGeneratedNames` can be passed to `ToCSharpString` to control whether the resulting string can contain compiler-generated names (otherwise, an exception will be thrown when encountering such a name). To illustrate this, let's have th C# compiler craft a closure type (also known as a display class).

In [ ]:
static class C
{
    public static Expression Expression
    {
        get
        {
            int x = 42;
            Expression<Func<int>> f = () => x;
            return f.Body;
        }
    }
}

This expression will be of the form `Member(Constant(closure), FieldInfo("x"))` in pseudo-code. We can now construct an expression tree whose C# syntax requires to reveal the type, e.g. a `default(T)` expression.

In [ ]:
var closureType = ((MemberExpression)C.Expression).Expression.Type;

var expr = Expression.Default(closureType);

Console.WriteLine(expr.ToCSharpString(allowCompilerGeneratedNames: true));

default(<>c__DisplayClass1_0)


Clearly this is not valid C# syntax. In fact, the whole purpose of the `<>__` naming prefix is to ensure that compiler-generated type names cannot conflict with identifiers introduced by user code, and to prevent user code from directly accessing these types. If we disable `allowCompilerGeneratedNames`, an exception is thrown instead.

In [ ]:
Console.WriteLine(expr.ToCSharpString(allowCompilerGeneratedNames: false));

System.InvalidOperationException: Compiler-generated C# type encountered: Submission#32+C+<>c__DisplayClass1_0
   at System.TypeExtensions.TypePrinter.Visit(Type type) in D:\Projects\Reaqtive\reaqtor\Nuqleon\Core\LINQ\Nuqleon.Linq.CompilerServices\TypeSystem\TypeExtensions.cs:line 711
   at System.TypeExtensions.ToCSharpString(Type type, Boolean useNamespaceQualifiedNames, Boolean useCSharpTypeAliases, Boolean disallowCompilerGeneratedTypes) in D:\Projects\Reaqtive\reaqtor\Nuqleon\Core\LINQ\Nuqleon.Linq.CompilerServices\TypeSystem\TypeExtensions.cs:line 665
   at System.Linq.Expressions.ExpressionCSharpPrinter.PrintType(Type type) in D:\Projects\Reaqtive\reaqtor\Nuqleon\Core\LINQ\Nuqleon.Linq.CompilerServices\Expressions\Diagnostics\ExpressionCSharpPrinter.cs:line 1152
   at System.Linq.Expressions.ExpressionCSharpPrinter.VisitDefault(DefaultExpression expression) in D:\Projects\Reaqtive\reaqtor\Nuqleon\Core\LINQ\Nuqleon.Linq.CompilerServices\Expressions\Diagnostics\ExpressionCSharpPri

Alternatively, the `ToCSharp` method can be used which returns a `CSharpExpression`, providing additional information about the tree, alongside a string representation. This additional information includes a table of `Constants` as well as a list of `GlobalParameters`. An example is shown below, where we have an opaque constant that doesn't have a C# representation, and an unbound variable.

In [ ]:
var expr = Expression.Add(Expression.Constant(DateTime.Now), Expression.Parameter(typeof(TimeSpan), "t"));

CSharpExpression csharpExpr = expr.ToCSharp();

Console.WriteLine(csharpExpr.Code);

Console.WriteLine("Globals:");

foreach (var global in csharpExpr.GlobalVariables)
{
    Console.WriteLine("  " + global);
}

Console.WriteLine("Constants:");

foreach (var constant in csharpExpr.Constants)
{
    Console.WriteLine("  " + constant);
}

__c0 + t


Globals:


  t


Constants:


  [__c0, 3/4/2021 12:32:16 PM]


## BURS

BURS stands for Bottom-Up Rewrite System and is based on the https://www.researchgate.net/publication/220752446_Simple_and_Efficient_BURS_Table_Generation paper. It provides for a table-driven weight-based approach to generate code. The tables contain rules which consist of patterns to recognize, a cost, and a rewrite rule.

This library generalizes the BURS mechanism to conversions between a generalized notion of trees, represented as `ITree<T>`. In this context, BURS is a table-driven way to build a compiler that translates between an `ITree<T>` to an `ITree<R>`. The special case of rewriting from an `ITree<T>` to an `ITree<T>` is a rule-driven optimizer.

### Building a rewriter

As an example, consider a first `ITree<T>` where `T` represents node information for a tree of some arithmetic language providing unary operators `+` and `-`, as well as binary operators `+`, `-`, `*`, and `%`. Building the whole `ITree<T>` implementation in the notebook gets a bit unweildy, so we'll just import a `SampleTrees` assembly that can be found in this repo, and is also used for testing.

In [ ]:
#r "..\SampleTrees\bin\Debug\net5.0\SampleTrees.dll"

We can now explore these tree types and try to craft an expression in this arithmetic expression language.

In [ ]:
using SampleTrees.Arithmetic;

var arithExpr = new Add(
    new Mul(
        new Add(
            new Mul(
                new Const(2),
                new Const(3)
            ),
            new Const(1)
        ),
        new Mul(
            new Const(4),
            new Const(5)
        )
    ),
    new Const(6)
);

Console.WriteLine(arithExpr);

Add(Mul(Add(Mul(Const(2), Const(3)), Const(1)), Mul(Const(4), Const(5))), Const(6))


Next, let's have a look at a second expression language, which we call numerical (for lack of a better name). It's similar to the arithmetic language, but has some different node types.

In [ ]:
using SampleTrees.Numerical;

var numExpr = new TimesPlus(
    new Inc(
        new Times(
            new Val(2),
            new Val(3)
        )
    ),
    new Times(
        new Val(4),
        new Val(5)
    ),
    new Val(6)
);

Console.WriteLine(numExpr);

TimesPlus(Inc(Times(2, 3)), Times(4, 5), 6)


Note how this second language has nodes like `Inc` and `TimesPlus` which are unary and ternary specialized operators. While this sample is quite academic in nature, it has resemblances to instructions in specialized hardware or highly optimized libraries for BLAS operations. Also note that the two expressions we built in the `Arithmetic` and `Numerical` languages are equivalent. Both compute the same result.

With BURS, we can write a rule-driven translator that takes an expression from the `Arithmetic` language and translates it to the `Numerical` language. Each rule has weights associates with it, and the rewriter tries to find a rewrite with the lowest weight. For example, we'd like `Add(Mul(2, 3), 1)` to use the specialized `Inc` rather than `Plus` with an operand of value `1`. Similarly, we'd like an `Add(Mul(a, b), c)` where `c` is different from `1` to leverage the specialized `TimesPlus` operator.

In the cell below, we'll go ahead and construct a BURS rewriter.

In [ ]:
using System.IO;
using System.Linq.CompilerServices;

var logger = new StringWriter();

var burw = new BottomUpRewriter<ArithExpr, ArithNodeType, NumExpr, ArithWildcardFactory>
{
    // Leaf nodes
    Leaves =
    {
        { (Const c) => new Val(c.Value), 1 },
    },

    // Tree patterns
    Rules =
    {
        { (l, r) => new Add(l, r), (l, r) => new Plus(l, r), 2 },
        { (l, r) => new Mul(l, r), (l, r) => new Times(l, r), 3 },
        { (a, b, c) => new Add(new Mul(a, b), c), (a, b, c) => new TimesPlus(a, b, c), 4 },
        { x => new Add(x, new Const(1)), x => new Inc(x), 1 },
    },

    Log = logger
};

The most interesting bit is the rules table which maps patterns on the left in the source language to productions on the right in the target language, with an associated weight. The leaf nodes collection tells the rewriter how to take nullary leaf nodes from the source to the target language. We also hooked up a logger to see the rewriter in action.

First, we can go ahead an inspect the rewriter's internal tables using a `DebugView` property.

In [ ]:
Console.WriteLine(burw.DebugView);

WILDCARDS:
  [l : |N, 0]

LEAVES:
  c => new Val(c.Value)  IF  node => (node Is Const)  (1$)

CONSTANTS:
  [Const(1), 4]

PATTERNS:
  [l, 0]
  [Add(l, r), 1]
  [Mul(l, r), 2]
  [Add(Mul(a, b), c), 3]
  [Const(1), 4]
  [Add(x, Const(1)), 5]

MAPS:
  Add
    (0, 0) --> 1
    (0, 4) --> 5
    (2, 0) --> 3
  Mul
    (0, 0) --> 2

FINAL:
  -2 - c => new Val(c.Value)  IF  node => (node Is Const)  (1$)
   1 - (l, r) => new Add(l, r) --(2$)--> (l, r) => new Plus(l, r)
   2 - (l, r) => new Mul(l, r) --(3$)--> (l, r) => new Times(l, r)
   3 - (a, b, c) => new Add(new Mul(a, b), c) --(4$)--> (a, b, c) => new TimesPlus(a, b, c)
   5 - x => new Add(x, new Const(1)) --(1$)--> x => new Inc(x)




This may look quite intimidating but it encodes the different rules in a bottom-up fashion. As a source tree is given to the rewriter, it starts by matching patterns in the rule tables starting from the leaf nodes. As it works it way up the tree, it tries to match bigger patterns and calculates the total weight of all candidate matches. When the tree is fully covered, it tries to minimize the cost by selecting the match with the lowest weight and then applies the rewrite rules. A concrete example will make this clear, by running our `arithExpr` through the rewriter.

In [ ]:
var numExpr = burw.Rewrite(arithExpr);

Console.WriteLine(numExpr);

TimesPlus(Inc(Times(2, 3)), Times(4, 5), 6)


Note we did get a tree that uses `TimesPlus` and `Inc` rather than less optimal patterns. Remember that we hooked up a logger to the rewriter, which we can now use to see the steps taken by the rewriter.

In [ ]:
Console.WriteLine(logger.ToString());

INPUT:
  Add(
    Mul(
      Add(
        Mul(
          Const(2), 
          Const(3)
        ), 
        Const(1)
      ), 
      Mul(
        Const(4), 
        Const(5)
      )
    ), 
    Const(6)
  )

LABELED:
  [0*, 1! (17$), 3! (16$)] Add(
    [0*, 2! (14$)] Mul(
      [0*, 1! (8$), 3! (7$), 5! (6$)] Add(
        [0*, 2! (5$)] Mul(
          [0*, -2! (1$)] Const(2), 
          [0*, -2! (1$)] Const(3)
        ), 
        [0*, 4 (0$), -2! (1$)] Const(1)
      ), 
      [0*, 2! (5$)] Mul(
        [0*, -2! (1$)] Const(4), 
        [0*, -2! (1$)] Const(5)
      )
    ), 
    [0*, -2! (1$)] Const(6)
  )

RESULT:
  TimesPlus(
    Inc(
      Times(
        2, 
        3
      )
    ), 
    Times(
      4, 
      5
    ), 
    6
  )




Describing the full inner workings of BURS is outside the scope of this notebook, but suffice it to say that the labeled tree shown above shows the different rules that match each of the nodes in the tree, with an associated cost (denoted by the `$` sign representing the total cost of the rewrite). For example, the top-level node has annotations:

```
0*, 1! (17$), 3! (16$)
```

The `*` refers to a wildcard, meaning this node could be matched in a bigger parent tree. The `1!` and `3!` are final productions that correspond to using `Plus` or `TimesPlus`, as can be seen from the `DebugView` we dumped earlier. The associated costs are `17$` and `16$`, which means that the second `TimesPlus` option is cheaper, and thus the rewriter follwed that rule to rewrite the expression.

### Building an optimizer

BURS can also be used to write rule-driven optimizers, which are merely rewriters where the input and output languages are the same. Unlike rewriters between different languages, the rule table of an optimizer does not have to cover all possible input patterns. Instead, an optimizer can simply express some patterns it knows to optimize. To illustrate this, we'll use yet another language, called `Logic` for Boolean-valued expressions.

In [ ]:
using SampleTrees.Logic;

var logicExpr = !(!BoolConst.True & !BoolConst.False);

Console.WriteLine(logicExpr);

Not(And(Not(True), Not(False)))


Obviously this can be reduced to a single constant `true` or `false` value, but it illustrates a number of possible rewrite rules. For example, `!true` can become `false`, but a more complex pattern of the form `!(!a & !b)` can also be turned into `a | b` using De Morgan's law. Once more, we can construct a table-driven rewriter that captures various rewrite rules.

In [ ]:
var logger = new StringWriter();

var burw = new BottomUpOptimizer<LogicExpr, LogicNodeType, LogicWildcardFactory>
{
    // Leaf nodes
    Leaves =
    {
        { (BoolConst b) => b, 1 },
    },

    Rules =
    {
        // Tree patterns
        { () => !BoolConst.True, () => BoolConst.False, 1 },
        { () => !BoolConst.False, () => BoolConst.True, 1 },
        { p => !!p, p => p, 2 },

        { p => p & BoolConst.True, p => p, 2 },
        { p => p & BoolConst.False, p => BoolConst.False, 2 },
        { p => BoolConst.True & p, p => p, 2 },
        { p => BoolConst.False & p, p => BoolConst.False, 2 },

        { p => p | BoolConst.True, p => BoolConst.True, 2 },
        { p => p | BoolConst.False, p => p, 2 },
        { p => BoolConst.True | p, p => BoolConst.True, 2 },
        { p => BoolConst.False | p, p => p, 2 },

        { (p, q) => !(!p & !q), (p, q) => p | q, 1 },
        { (p, q) => !(!p | !q), (p, q) => p & q, 1 },
    },

    Log = logger
};

Note we have rules for negation of constants, double negation, use of `&` and `|` where one operand is a constant, but also the De Morgan rules at the bottom. Let's feed the expression from above to the rewriter and see what happens.

In [ ]:
var optimizedLogicExpr = burw.Optimize(logicExpr);

Console.WriteLine(optimizedLogicExpr);

True


This time, we called `Optimize` rather than `Rewrite` which keeps performing rewrites until the tree no longer changes. If we were to just call `Rewrite` (left as an exercise), we may end up with a tree that's not yet fully optimized.

Let's now have a look at the rewrites that took place, by inspecting the `logger`.

In [ ]:
Console.WriteLine(logger.ToString());

INPUT:
  Not(
    And(
      Not(
        True
      ), 
      Not(
        False
      )
    )
  )

LABELED:
  [4*, 5 (0$), 16! (3$)] Not(
    [4*, 15 (0$)] And(
      [4*, 5 (0$), 1! (1$)] Not(
        [4*, 0 (0$), -2! (1$)] True
      ), 
      [4*, 5 (0$), 3! (1$)] Not(
        [4*, 2 (0$), -2! (1$)] False
      )
    )
  )

RESULT:
  Or(
    True, 
    False
  )

INPUT:
  Or(
    True, 
    False
  )

LABELED:
  [4*, 13! (3$), 12! (3$)] Or(
    [4*, 0 (0$), -2! (1$)] True, 
    [4*, 2 (0$), -2! (1$)] False
  )

RESULT:
  True

INPUT:
  True

LABELED:
  [4*, 0 (0$), -2! (1$)] True

RESULT:
  True




We can see that the rewriter was called more than once. First, our tree got rewritten using rules for Boolean negation at the leafs (`!true` became `false`, and vice versa), and the Morgan's law was applied at the top, turning the `Not(And(...))` into an `Or`. The second optimization pass was then faced with an `Or` with a `true` operand, which got ultimately reduced to `true`.

### Applying BURS to LINQ expression trees

In order to use BURS with .NET expression trees, one has to first convert expression trees to the `ITree<T>` interface type that BURS operates on. The library comes with a `ToExpressionTree` extension method that can be used to achieve this.

In [ ]:
var expr = Expression.And(Expression.Constant(true), Expression.Constant(false));

var exprTree = expr.ToExpressionTree();

Console.WriteLine(exprTree);

And(Constant[(bool)True](), Constant[(bool)False]())


Note that the `ToString` representation of an `ExpressionTree` (which is an `ITree<ExpressionTreeNode>`) is a little funny looking because it rigorously prints the node's kind followed by the children in between `(` and `)`. Leaf nodes like constants have no children and end up with a `()` at the end.

Once we have an `ExpressionTree` which implements `ITree<T>` we can start to use BURS to perform rewrites, e.g. to translate an expression tree to some target language. BURS has been used to build table-driven query providers (e.g. going from expression trees to some `ITree<SqlNode>` for translation to SQL) and optimizers. Mileage varies depending on the complexity of the rule matching involved. Further extensions of BURS have been written in spin-off projects, modeling type system checks (e.g. how does a rule involving a method invocation `object.Equals(object)` relate to rules that involve an override of this virtual method on a more derived type?), supporting additional predicates to drive the rule selection process, and dynamic computation of weights.

## Miscellaneous utilities

In this final section of the notebook, we'll look at some remaining utilities that are provided by the library.

### `ReflectionHelpers`

This type provides a set of `InfoOf` methods that obtain a `MemberInfo` from an expression tree. This is a mechanism akin to `typeof` for types but targeting methods, properties, fields, and constructors instead (much like a hypothethical C# `infoof` operator could do). For example:

In [ ]:
Console.WriteLine((MethodInfo)ReflectionHelpers.InfoOf((string s) => s.Substring(0, 1)));
Console.WriteLine((PropertyInfo)ReflectionHelpers.InfoOf(() => DateTime.Now));
Console.WriteLine((ConstructorInfo)ReflectionHelpers.InfoOf(() => new TimeSpan(0, 0, 0)));

System.String Substring(Int32, Int32)


System.DateTime Now


Void .ctor(Int32, Int32, Int32)


### `RuntimeCompiler`

The runtime compiler uses `System.Reflection.Emit` to build anonymous types, closure types, and so-called record types. The resulting types are often used in expression trees. For example, one could erase nominal types (such as a `Person` type) for structurally identical types (e.g. a record containing a `string Name` and `int Age`), which then case be used to serialize types by their shape rather than by their name.

#### Anonymous types

Runtime-generated anonymous types are analogous to C# 3.0 and VB 9.0 anonymous types. Both flavors can be built using different overloads of `CreateAnonymousType`:

```csharp
public static Type CreateAnonymousType(IEnumerable<KeyValuePair<string, Type>> properties);
public static Type CreateAnonymousType(IEnumerable<StructuralFieldDeclaration> properties);

public static Type CreateAnonymousType(IEnumerable<KeyValuePair<string, Type>> properties, params string[] keys);
public static Type CreateAnonymousType(IEnumerable<StructuralFieldDeclaration> properties, params string[] keys);
```

Overloads with `StructuralFieldDeclaration` support adding custom attributes to the generated properties. The difference between overloads that lack a `keys` parameter versus the ones that have one has to do with the properties that participate in the implementation for `Equals` and `GetHashCode`. Being able to specify particular properties as "keys" matches the design of anonymous types in Visual Basic.

An example is shown below:

In [ ]:
var anon = RuntimeCompiler.CreateAnonymousType(new StructuralFieldDeclaration[]
    {
        new("Name", typeof(string)),
        new("Age", typeof(int))
    });

Console.WriteLine(anon);

var person1 = Activator.CreateInstance(anon, new object[] { "Bart", 21 });

Console.WriteLine(person1);

var person2 = Activator.CreateInstance(anon, new object[] { "Bart", 21 });

Console.WriteLine(person2);

Console.WriteLine($"Are equal? {person1.Equals(person2)}");

<>f__AnonymousType1


{ Name = Bart, Age = 21 }


{ Name = Bart, Age = 21 }


Are equal? True


#### Closure types

Closure types are simply classes that declare a bunch of fields. To create a closure type at runtime, use the `CreateClosureType` method:

```csharp
public static Type CreateClosureType(IEnumerable<KeyValuePair<string, Type>> fields);
```

An example is shown below:

In [ ]:
var closure = RuntimeCompiler.CreateClosureType(new KeyValuePair<string, Type>[]
    {
        new("x", typeof(int)),
        new("b", typeof(bool))
    });

Console.WriteLine(closure);

dynamic d = Activator.CreateInstance(closure);
d.x = 42;
d.b = true;

<>c__DisplayClass1


#### Record types

The notion of record types in this library predates C# 9.0's record types by almost a decade, so their characteristics are quite different. Record types in this library are classes that are similar to anonymous types but provide control over the implementation of equality (value versus reference equality). Furthermore, they support assignment to properties. Use the `CreateRecordType` method to create them:

```csharp
public static Type CreateRecordType(IEnumerable<KeyValuePair<string, Type>> properties, bool valueEquality);
public static Type CreateRecordType(IEnumerable<StructuralFieldDeclaration> properties, bool valueEquality);
```

An example is shown below:

In [ ]:
var record = RuntimeCompiler.CreateRecordType(new StructuralFieldDeclaration[]
    {
        new("Name", typeof(string)),
        new("Age", typeof(int))
    }, valueEquality: true);

Console.WriteLine(record);

dynamic person1 = Activator.CreateInstance(record);
person1.Name = "Bart";
person1.Age = 21;

Console.WriteLine(person1);

dynamic person2 = Activator.CreateInstance(record);
person2.Name = "Bart";
person2.Age = 21;

Console.WriteLine(person2);

Console.WriteLine($"Are equal? {person1.Equals(person2)}");

<>a__RecordType1


{ Name = Bart, Age = 21 }


{ Name = Bart, Age = 21 }


Are equal? True


#### `Define*` method variants

In addition to the `Create*` methods illustrated above, variants with the `Define*` prefix are provided as well. Rather than returning a `Type`, these accept a `TypeBuilder` to define the type on. These variants are useful when trying to build recursive types (i.e. there's a cycle between declarations and uses of types), because one can use `TypeBuilder` instances for the types of the properties on the anonymous or record type being constructed. Furthermore, one could extend these types with custom members (e.g. a closure type could have instance members associated with it that operate on the state captured in fields). Once all types have been defined, the user can then call `CreateType` on the `TypeBuilder` instances. Examples of types with cycles are:

```csharp
// A -> A
class A
{
    public A Next { get; set; }
}

// B -> C
class B
{
    public C C { get; set; }
}

// C -> B
class C
{
    public B B { get; set; }
}
```

As an example, a structurally equivalent pair of record types for `B` and `C` could be built as follows.

In [ ]:
using System.Reflection.Emit;

var compiler = new RuntimeCompiler();

TypeBuilder tb = compiler.GetNewRecordTypeBuilder();
TypeBuilder tc = compiler.GetNewRecordTypeBuilder();

compiler.DefineRecordType(tb, new KeyValuePair<string, Type>[] { new("C", tc) }, valueEquality: false);
compiler.DefineRecordType(tc, new KeyValuePair<string, Type>[] { new("B", tb) }, valueEquality: false);

Type b = tb.CreateType();
Type c = tc.CreateType();

dynamic objB = Activator.CreateInstance(b);
dynamic objC = Activator.CreateInstance(c);

objB.C = objC;
objC.B = objB;

Console.WriteLine(objB.GetHashCode());
Console.WriteLine(objC.GetHashCode());
Console.WriteLine(objB.C.GetHashCode());
Console.WriteLine(objB.C.B.GetHashCode());

16541276


5663325


5663325


16541276


Note we use `valueEquality` set to `false` in the example above. This avoids implementations like `Equals` and `GetHashCode` from going in a cycle. Using methods like `ToString` on cyclic types like this one is fraught with danger. If you want to exit the notebook with some fireworks, run the following cell to see why (**at your own risk**).

In [ ]:
Console.WriteLine(objB.ToString());